# Analyse über Einflussfaktoren zum Stromverbrauch im Versorgungsgebiet der CKW AG auf Ebene der Gemeinde
*Business Intelligence and Analytics, MScWI FS24 Hochschule Luzern*
- **Noemi Rohner**
- **Mizgin Turunc**
- **Jan Leuenberger**
- **Lukas Bucheli**


Die CKW AG ist ein zentralschweizer Energieunternehmen, welches sich aufgrund der Energiestrategie 2050 verpflichten musste bis 2027 sämtliche traditionellen Zähler durch Smartmeter zu ersetzen. Sie sollen Teil des Smart Grid werden. Mit einem Smart Grid erhofft man sich eine effizientere Energieversorgung. Die Smartmeter sind ein integrierter Bestandteil des Smart Grid und ermöglichen dadurch neue Funktionen wie intelligente Steuerungen. Besonders durch die aufkommende dezentralisierte Energieerzeugung mit erneuerbaren Energien bei privaten Haushalten, müssen intelligent gesteuert werden, damit das Stromnetz ausbalanciert bleibt. Mit diesen Smartmeter wird auch der Echtzeit-Zugriff auf die Zählerdaten für den Endkunden ermöglicht. Dadurch erhofft sich das Bundesamt für Energie (BFE) einen bewussteren Umgang mit dem Strom und dadurch auch diesen zu sparen (BFE, 2021 [Online-Quelle](https://www.bfe.admin.ch/bfe/de/home/versorgung/stromversorgung/stromnetze/smart-grids.html)).

## Fragestellung
In dieser Arbeit wird folgende Fragestellung versucht zu beantworten:
> Was sind relevante Einflussfaktoren auf Gemeindeebene in Bezug auf den Stromverbrauch im Einzugsgebiet der CKW?

## Aufbau der Jupyter Notebooks
Aufgrund der umfassenden Datenmenge und dem Einsatz einer SQLite Datenbank wurde entschieden, die Arbeit in mehrere Jupyter Notebooks aufzuteilen:
- **bina_dataimport.ipynb**, In diesem Notebook werden die einzelnen verwendeten Datasets analysiert und in die Datenbank geladen.
- **data_analysis.ipynb**, In diesem Notebook folgt die vernetzte Analyse der Daten.

# Datenquellen und Datenbank
In diesem Notebook werden die Daten aus der im Import Notebook erstellten Datenbank geladen. Dazu muss die Datenbank zuvor mit dem Notebook erstellt worden sein.


## Eingesetzte Module
Für dieses Notebook werden folgende Module eingesetzt:
- __sqlite3__
    Wird für die Datenbank verwendet
- __warnings__
    Um FutureWarnings zu unterdrücken, welche in der aktuellen Pandas Version nicht relevant sind
- __pandas__
    Wird gebraucht um die CSV Dateien zu lesen und als Dataframe in die SQLite Datenbank zu laden
- __geopandas__
    Wird gebraucht um Geografische Karten darzustellen
- __math__
    Wird verwendet für mathematische Funktionen
- __seaborn__
    Wird für Daten-Visualisierungen verwendet. Basiert auf matplotlib
- __matplotlib__
    Wird verwendet für statistische Berechnungen sowie Darstellungen
- __plotly__
    Wird verwendet für statistische Berechnungen sowie Darstellungen
- __numpy__
    Wird benötigt um die NaN Werte in den Dataframes mit NULL zu ersetzen
- __statsmodels__
    Wird eingesetzt 
- __pmdarima__
    Wird eingesetzt um leere Datensätze in Timeseries aufzufüllen
- __tbats__
    Wird für das Forecasting von Timeseries verwendet 
- __sklearn__
    Wird für das Machine Learning eingesetzt 

In [ ]:
# Load GEOPANDAS library
import os, sqlite3
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import geopandas as gpd

import math
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import numpy as np

import statsmodels.api as sm
from statsmodels.tsa.stl.mstl import MSTL
from statsmodels.tsa.stattools import adfuller

import pmdarima as pm
from tbats import TBATS, BATS

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from statsmodels.tsa.seasonal import seasonal_decompose

# Settings for clean html export
pio.renderers.default = "plotly_mimetype+notebook"

## Funktionen
Nachfolgend werden mehrere Funktionen und Klassen definiert, welche eine Übersetzung des Schweizer Koordinatensystems in das globale Koordinatensystem ermöglichen
Quelle: Swisstopo, https://github.com/ValentinMinder/Swisstopo-WGS84-LV03/tree/master

In [ ]:
class GPSConverter(object):
    '''
    GPS Converter class which is able to perform convertions between the 
    CH1903 and WGS84 system.
    '''
    # Convert CH y/x/h to WGS height
    def CHtoWGSheight(self, y, x, h):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        h = (h + 49.55) - (12.60 * y_aux) - (22.64 * x_aux)
        return h

    # Convert CH y/x to WGS lat
    def CHtoWGSlat(self, y, x):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        lat = (16.9023892 + (3.238272 * x_aux)) + \
              - (0.270978 * pow(y_aux, 2)) + \
              - (0.002528 * pow(x_aux, 2)) + \
              - (0.0447 * pow(y_aux, 2) * x_aux) + \
              - (0.0140 * pow(x_aux, 3))
        # Unit 10000" to 1" and convert seconds to degrees (dec)
        lat = (lat * 100) / 36
        return lat

    # Convert CH y/x to WGS long
    def CHtoWGSlng(self, y, x):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        lng = (2.6779094 + (4.728982 * y_aux) + \
               + (0.791484 * y_aux * x_aux) + \
               + (0.1306 * y_aux * pow(x_aux, 2))) + \
              - (0.0436 * pow(y_aux, 3))
        # Unit 10000" to 1" and convert seconds to degrees (dec)
        lng = (lng * 100) / 36
        return lng

    # Convert decimal angle (° dec) to sexagesimal angle (dd.mmss,ss)
    def DecToSexAngle(self, dec):
        degree = int(math.floor(dec))
        minute = int(math.floor((dec - degree) * 60))
        second = (((dec - degree) * 60) - minute) * 60
        return degree + (float(minute) / 100) + (second / 10000)

    # Convert sexagesimal angle (dd.mmss,ss) to seconds
    def SexAngleToSeconds(self, dms):
        degree = 0
        minute = 0
        second = 0
        degree = math.floor(dms)
        minute = math.floor((dms - degree) * 100)
        second = (((dms - degree) * 100) - minute) * 100
        return second + (minute * 60) + (degree * 3600)

    # Convert sexagesimal angle (dd.mmss) to decimal angle (degrees)
    def SexToDecAngle(self, dms):
        degree = 0
        minute = 0
        second = 0
        degree = math.floor(dms)
        minute = math.floor((dms - degree) * 100)
        second = (((dms - degree) * 100) - minute) * 100
        return degree + (minute / 60) + (second / 3600)

    # Convert WGS lat/long (° dec) and height to CH h
    def WGStoCHh(self, lat, lng, h):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        h = (h - 49.55) + (2.73 * lng_aux) + (6.94 * lat_aux)
        return h

    # Convert WGS lat/long (° dec) to CH x
    def WGStoCHx(self, lat, lng):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        x = ((200147.07 + (308807.95 * lat_aux) + \
              + (3745.25 * pow(lng_aux, 2)) + \
              + (76.63 * pow(lat_aux,2))) + \
             - (194.56 * pow(lng_aux, 2) * lat_aux)) + \
            + (119.79 * pow(lat_aux, 3))
        return x

    # Convert WGS lat/long (° dec) to CH y
    def WGStoCHy(self, lat, lng):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        y = (600072.37 + (211455.93 * lng_aux)) + \
            - (10938.51 * lng_aux * lat_aux) + \
            - (0.36 * lng_aux * pow(lat_aux, 2)) + \
            - (44.54 * pow(lng_aux, 3))
        return y

    def LV03toWGS84(self, east, north, height):
        '''
        Convert LV03 to WGS84 Return a array of double that contain lat, long,
        and height
        '''
        d = []
        d.append(self.CHtoWGSlat(east, north))
        d.append(self.CHtoWGSlng(east, north))
        d.append(self.CHtoWGSheight(east, north, height))
        return d

    def WGS84toLV03(self, latitude, longitude, ellHeight):
        '''
        Convert WGS84 to LV03 Return an array of double that contaign east,
        north, and height
        '''
        d = []
        d.append(self.WGStoCHy(latitude, longitude))
        d.append(self.WGStoCHx(latitude, longitude))
        d.append(self.WGStoCHh(latitude, longitude, ellHeight))
        return d

## Daten vorbereiten
### Geodaten
Für die grafische Darstellung verschiedener Auswertungen auf der Schweizer Karte werden entsprechende Geodaten benötigt, welche anschliessend mit der BFS-ID mit Auswertungen verknüpft und so dargestellt werden.

In [ ]:
# read Swiss Cantons Geometry Data (as GeoFrame)
geofilePATH = 'https://raw.githubusercontent.com/sawubona-repo/BINA-FS24-WORK/master/zDiversExamples/Notebook-GeoMapping/DATA/'
geofileNAME = 'ch-municipalities.geojson'

# Read GeoJSON geometry data into geopandas GeoDataFrame
raw_geodf = gpd.read_file(geofilePATH+geofileNAME)

raw_geodf.info()

### Datenbank
Um die im anderen Notebook erstellte Datenbank auszulesen, wird diese nachfolgend hinzugefügt.

In [ ]:
dbfile = './DATA/BINA_DATA.db'

# Test if the database file is available in the colab workspace
if os.path.exists(dbfile):
    # Create database (file) and Open a (SQL) connection 
    connection = sqlite3.connect(dbfile)
    # Create a data cursor to exchange information between Python and SQLite
    cursor = connection.cursor()
else:
    print("Angegebene Database wurde nicht gefunden")
    # Create database (file) and Open a (SQL) connection 
    connection = sqlite3.connect(dbfile)
    # Create a data cursor to exchange information between Python and SQLite
    cursor = connection.cursor()
    #sys.exit(0)

# Smartmeter
Eine einfache Auswertung der maximalen Anzahl Smartmeter pro Gemeinde in den Daten soll aufzeigen, wie viele Smartmeter bereits zum Einsatz kommen. Dabei zeigt sich, dass diese Zahlen sehr unterschiedlich sind.

In [ ]:
anzSmartmeter = pd.read_sql_query("SELECT MAX(anzMeter) anz, bfsID FROM smartmeter s JOIN plzBfsMapping p ON s.plz = p.plz GROUP BY bfsID;", connection)
joined_anzSmartmeter = pd.merge(raw_geodf, anzSmartmeter, left_on="gemeinde.BFS_NUMMER", right_on="bfsID")
anzSmartmeter = joined_anzSmartmeter

# Create Choropleth GeoMap with Population Data (Feature "zAnteil_Schweizer")
fig = px.choropleth_mapbox(
    anzSmartmeter,
    geojson=anzSmartmeter.geometry,
    locations=anzSmartmeter.index,
    color='anz',                                   # define feature variable
    color_continuous_scale=px.colors.diverging.Geyser,           # define color palette
    labels={'anz':'Anzahl Smartmeter', 'gemeinde.NAME':'Gemeinde'},

    hover_name='gemeinde.NAME',                                       # define mouse over infos
    hover_data={'gemeinde.NAME':True, 'anz':True},
    opacity=0.5,
    center=dict(lat=47.05048, lon=8.30635),                      # set lucerne as map center
    zoom=8,
    mapbox_style="carto-positron"                                # other option "open-street-map"
)

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.show()

Um herauszufinden, wie weit der Ausbau mit Smartmetern bereits fortgeschritten ist, kann die Anzahl Smartmeter mit der Anzahl Haushalten pro Gemeinde in Verbindung gebracht werden. Hier zeigt sich jedoch, dass bei einigen Gemeinden ein Wert von >1 erreicht wird. Dies deutet darauf hin, dass nebst privaten Haushalten auch Betriebe bereits damit ausgestattet wurden, welche nicht in die Kategorie "Grossverbraucher" fallen.

In [ ]:
smartmeterPerHousehold = pd.read_sql_query("SELECT ROUND(MAX(s.anzMeter) / d.value, 2) AS anz, d.bfsID FROM smartmeter s JOIN plzBfsMapping p ON s.plz = p.plz JOIN demoValue d ON p.bfsID = d.bfsID WHERE d.indicator = 'Ind_01_13' GROUP BY d.bfsID, d.value;", connection)
joined_smartmeterPerHousehold = pd.merge(raw_geodf, smartmeterPerHousehold, left_on="gemeinde.BFS_NUMMER", right_on="bfsID")
smartmeterPerHousehold = joined_smartmeterPerHousehold

# Create Choropleth GeoMap with Population Data (Feature "zAnteil_Schweizer")
fig = px.choropleth_mapbox(
    smartmeterPerHousehold,
    geojson=smartmeterPerHousehold.geometry,
    locations=smartmeterPerHousehold.index,
    color='anz',                                   # define feature variable
    color_continuous_scale=px.colors.sequential.Greens,           # define color palette
    labels={'anz':'Anzahl Smartmeter pro Haushalt', 'gemeinde.NAME':'Gemeinde'},

    hover_name='gemeinde.NAME',                                       # define mouse over infos
    hover_data={'gemeinde.NAME':True, 'anz':True},
    opacity=0.5,
    center=dict(lat=47.05048, lon=8.30635),                      # set lucerne as map center
    zoom=8,
    mapbox_style="carto-positron"                                # other option "open-street-map"
)

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.update_layout(title_text='Smartmeter pro Haushalt')
fig.show()

In [ ]:
df = pd.read_sql_query("SELECT s.plz, SUM(valueKwh / anzMeter) as sumValuePerPlz, p.bfsID FROM smartmeter s JOIN plzBfsMapping p ON s.plz = p.plz GROUP BY s.plz ORDER BY sumValuePerPlz DESC;", connection)
geodf = pd.merge(raw_geodf, df, left_on="gemeinde.BFS_NUMMER", right_on="bfsID")

# Create Choropleth GeoMap
fig = px.choropleth_mapbox(
    geodf,
    geojson=geodf.geometry,
    locations=geodf.index,
    color='sumValuePerPlz',                                   # define feature variable
    color_continuous_scale=px.colors.diverging.Geyser,           # define color palette
    labels={'sumValuePerPlz':'Stromverbrauch pro Smart Meter'},

    hover_name='gemeinde.NAME',                                       # define mouse over infos
    hover_data={'gemeinde.NAME':True, 'sumValuePerPlz':True},
    opacity=0.5,
    center=dict(lat=47.05048, lon=8.30635),                      # set lucerne as map center
    zoom=8,
    mapbox_style="carto-positron"                                # other option "open-street-map"
)

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.show()

In [ ]:
# Get Data
df = pd.read_sql_query("SELECT (a.anzMeter / p.value) AS smartmeterPerPopulation, a.bfsID FROM (SELECT MAX(anzMeter) AS anzMeter, bfsID FROM smartmeter s JOIN plzBfsMapping m ON s.plz = m.plz GROUP BY bfsID) a JOIN population p ON a.bfsID = p.bfsID;", connection)

# Merge Data with geo information
geodf = pd.merge(raw_geodf, df, left_on="gemeinde.BFS_NUMMER", right_on="bfsID")

# Create Choropleth GeoMap with Population Data (Feature "zAnteil_Schweizer")
fig = px.choropleth_mapbox(
    geodf,
    geojson=geodf.geometry,
    locations=geodf.index,
    color='smartmeterPerPopulation',                                   # define feature variable
    color_continuous_scale=px.colors.diverging.Geyser,           # define color palette
    labels={'smartmeterPerPopulation':'Smartmeter pro Einwohner'},

    hover_name='gemeinde.NAME',                                       # define mouse over infos
    hover_data={'gemeinde.NAME':True, 'smartmeterPerPopulation':True},
    opacity=0.5,
    center=dict(lat=47.05048, lon=8.30635),                      # set lucerne as map center
    zoom=8,
    mapbox_style="carto-positron"                                # other option "open-street-map"
)

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.show()

In [ ]:
solarPower = pd.read_sql_query("SELECT a.sumKwh as sumKwhUsed, a.bfsID, a.lawCityName, o.sumTotalPower as sumKwhSolar FROM (SELECT SUM(valueKwh / anzMeter) AS sumKwh, bfsID, lawCityName FROM smartmeter s JOIN plzBfsMapping m ON s.plz = m.plz GROUP BY bfsID) a JOIN solarPlantsLUbfsId o ON a.bfsID = o.bfsID;", connection)

# Hochdorf aus Auswertung entfernen, da Daten fehlerhaft
solarPower = solarPower[solarPower.bfsID != 1031]

sns.scatterplot(x='sumKwhUsed', y='sumKwhSolar', data=solarPower).set(xlabel='Usage per Smartmeter (kW)', ylabel='Total Solar Power')
plt.show()

Um mögliche Zusammenhänge zwischen Stromverbrauch pro Smartmeter und der Bevölkerungsanzahl zu finden, werden nachfolgend der Verbrauch pro Smartmeter und die Bevölkerungsanzahl gegenübergestellt. Farblich eingefärbt ist zusätzlich die Anzahl Unternehmen pro Gemeinde ersichtlich. Es zeigt sich, dass kein Zusammenhang zwischen Bevölkerungsgrösse und Stromverbrauch pro Smartmeter zu erkennen ist. Eine Aussage, dass kleinere Gemeinden mehr oder weniger Strom pro Smartmeter verbrauchen als grosse, ist also nicht möglich. Einzig ein Zusammenhang zwischen Bevölkerungsgrösse und Anzahl Unternehmen ist zu erkennen, was darauf schliessen lässt, dass grössere Gemeinden mit mehr Einwohner auch mehr Unternehmen beheimaten.

In [ ]:
kwhPopulation = pd.read_sql_query("SELECT SUM(s.valueKwh / s.anzMeter) AS sumKwhUsed, m.bfsID, m.lawCityName, o.value AS population, b.value AS industry FROM smartmeter s JOIN plzBfsMapping m ON s.plz = m.plz JOIN demoValue o ON m.bfsID = o.bfsID AND o.indicator = 'Ind_01_01' JOIN demoValue b ON m.bfsID = b.bfsID AND b.indicator = 'Ind_06_07' GROUP BY m.bfsID, m.lawCityName, o.value, b.value;", connection)

# Hochdorf aus Auswertung entfernen, da Daten fehlerhaft
kwhPopulation = kwhPopulation[kwhPopulation.bfsID != 1031]
# Stadt Luzern aus Auswertung entfernen, da starke Abweichung bei Bewohner
kwhPopulation = kwhPopulation[kwhPopulation.bfsID != 1061]

fig, axes = plt.subplots(1, 2, figsize=(16,5))
fig.suptitle('Zusammenhang Verbrauch per Smartmeter & demografische Daten')
sns.scatterplot(ax = axes[0], x='sumKwhUsed', y='population', hue='industry', data=kwhPopulation).set(xlabel='Usage per Smartmeter (kW)', ylabel='Population')
sns.scatterplot(ax = axes[1], x='sumKwhUsed', y='industry', data=kwhPopulation).set(xlabel='Usage per Smartmeter (kW)', ylabel='Industry')

plt.show()

Eine reine Gegenüberstellung von Stromverbrauch pro Smartmeter und der Anzahl Unternehmen zeigt auch keine Zusammenhänge auf. Unternehmen scheinen also nicht primär für einen höheren Verbrauch pro Smartmeter verantwortlich zu sein.

Weitere, tiefere Analysen können womöglich noch Zusammenhänge zwischen Stromverbrauch pro Smartmeter und demografischen Daten aufzeigen. Dazu scheint jedoch die Anzahl der Gemeinden zu klein, um aussagekräftig zu sein und die Daten der Smartmeter weisen zu viele Ungenauigkeiten und Fehler auf. Stünden verlässliche Smartmeter Daten schweizweit zur Verfügung, könnten womöglich Zusammenhänge hergestell werden.

## Datenanalyse mit Meteodaten und Smartmeterdaten
In diesem Notebook geht es darum kombinierte Analysen auszuführen. Also wie stehen die Smartmeter Daten im Verhältnis zu Meteodaten.

### Datenanalyse Smartmeter und Sonnendaten
Zuerst werden die Daten aus der Datenbank geladen und gleich im korrekten Zeitformat gesichert. Aus der ersten Analyse ist bekannt, dass es 4 Meteostationen gibt, welche die Sonnenminuten gemessen haben. Von diesen haben zwei (MOA und LUZ) den gesamten Zeitraum zu Verfügung. Dies wird nochmals in der Grafik ersichtlich. Es ist auch erkennbar, dass eine Saisonalität (Sommer/Winter) vorhanden ist. Im Winter gibt es erkennbar weniger Sonnenminuten wie im Sommer. Die geladenen Daten beinhalten die aufsummierten täglichen Sonnenminuten pro Tag und BFS-Nummer.

In [ ]:
df_sun = pd.read_sql_query("SELECT SUM(meteoData.value) as 'sunMinDay', meteoData.dataTime, meteoData.meteoParameter, meteoStation FROM  meteoData WHERE meteoParameter == 'sre000h0' GROUP BY meteoStation, strftime('%Y-%m-%d', meteoData.dataTime)", connection)
df_sun['dataTime'] = pd.to_datetime(df_sun['dataTime'], format='%Y-%m-%d %H:%M')

plt.figure(figsize=(10, 6))
testPlot = sns.lineplot(x='dataTime', y='sunMinDay', hue='meteoStation', data=df_sun).set(title='Anz. Sonnenminuten pro Tag und Messstation', xlabel='Datum', ylabel='Anz. Sonnenminuten')
plt.xticks(rotation=90)
plt.show()

Als nächstes werden die Sonnenminuten und Zählerdaten pro BFS-Nummer geladen. Bei den Zählerdaten handelt es sich immer um den Durchschnittsverbrauch pro Gemeinde pro Zähler. Damit sind die Gemeinden untereinander vergleichbar. Mit der BFS-Nummer und dem Datum lassen sich die Daten verknüpfen.

In [ ]:
df_sun = pd.read_sql_query("SELECT * FROM sumSunMinutesPerDayBfsId", connection)
df_sun.head()

In [ ]:
df_power = pd.read_sql_query("SELECT * FROM dailySumSmartmeterBfsId", connection)
df_power.head()

In [ ]:
df_sunPower = pd.merge(df_sun, df_power, on=['time', 'bfsID'])
df_sunPower['time'] = pd.to_datetime(df_sunPower['time'], format='%Y-%m-%d')
df_sunPower.head()

### Erstanalyse am Beispiel der Gemeinde Meggen
Bevor eine Analyse über sämtliche Daten durchgeführt wird, wird die Gemeinde Meggen als Beispiel genommen. Es wird sichtbar, dass im Fall von Meggen drei Sonnenwerte fehlen. Diese müssen noch aufgefüllt werden um eine komplette Zeitreihe zu erhalten.

In [ ]:
df_sunPowerMeggen = df_sunPower.query("bfsID == 1063").copy()
df_sunPowerMeggen.info()
df_sunPowerMeggen.head()

Zuerst aber eine Grafik, welche auf der linken y-Achse wird der Verbrauch dargestellt und rechts die Sonnenminute. Bei der Betrachtung des Stromverbrauches erkennt man ebenfalls eine Saisonalität. Im Winter steigt der durchschnittliche Stromverbrauch und im Sommer nimmt dieser ab. Im Gegensatz zu der Sonne, welche zeitweise den Boden nicht erreicht (min = 0), wird permanent Strom verbraucht und ist nie 0.

In [ ]:
meggenPlot, ax1 = plt.subplots(figsize=(10, 6))
sns.lineplot(x='time', y='avgKwhConsum', data=df_sunPowerMeggen, ax=ax1).set(title='Vergleich des durchschnittlichen Stromverbrauches zu den Sonnenminute', xlabel='Datum', ylabel='Verbrauch in kWh')
ax2 = ax1.twinx()
sns.lineplot(data=df_sunPowerMeggen, x='time', y='sumSunDay', ax=ax2, color='red').set(ylabel='Anz. Sonnenminuten')
plt.xticks(rotation=90)
plt.show()

Für den Umgang mit den NA Werten, werden zunächst die unnötigen Spalten entfernt und kontrolliert, ob jedes Datum vorhanden ist.

In [ ]:
df_meggen = df_sunPowerMeggen.copy()
df_meggen.drop(columns=['meteoParameter', 'meteoStation', 'bfsID', 'lawCityName'], inplace=True)
df_meggen['time'] = pd.to_datetime(df_meggen['time'])
print('Fehlende Datum: ', pd.date_range(start = '2020-12-31', end='2024-03-05').difference(df_meggen['time']))

Für die spätere Kontrolle wie gut die Werte, zu den erzeugten Werten passen, wird eine Spalte erzeugt mit fehlenden Werten. Die fehlende Werte in der echten Spalten müssen entfernt werden, damit die spätere Kontrolle mit dem R-Squared funktioniert.

In [ ]:
df_meggen.set_index('time', inplace=True)
df_meggen.dropna(inplace=True)
df_meggen['sumSunDayRef'] = df_meggen['sumSunDay']
df_meggen['sumSunDayRef'] = df_meggen['sumSunDayRef'].sample(frac=0.85)
df_meggen.info()
df_meggen.head()

Nun werden verschiedene Methoden angewandt, die fehlenden Werte der Referenz-Spalte zu füllen. In der Übersicht erkennt man, dass jede Spalte, bis auf die Referenzspalte, eine vollständige Spalte besitzt.

In [ ]:
df_meggen = df_meggen.assign(FillMean=df_meggen.sumSunDayRef.fillna(df_meggen.sumSunDayRef.mean()))
df_meggen = df_meggen.assign(FillMedian=df_meggen.sumSunDayRef.fillna(df_meggen.sumSunDayRef.median()))
df_meggen = df_meggen.assign(RollingMean=df_meggen.sumSunDayRef.fillna(df_meggen.sumSunDayRef.rolling(24, min_periods=1).mean()))
df_meggen = df_meggen.assign(RollingMedian=df_meggen.sumSunDayRef.fillna(df_meggen.sumSunDayRef.rolling(24, min_periods=1).median()))
df_meggen = df_meggen.assign(InterpolationLinear=df_meggen.sumSunDayRef.interpolate(method='linear'))
df_meggen = df_meggen.assign(InterpolationQuadratic=df_meggen.sumSunDayRef.interpolate(method='quadratic'))
df_meggen = df_meggen.assign(InterpolationCubic=df_meggen.sumSunDayRef.interpolate(method='cubic'))
df_meggen = df_meggen.assign(InterpolationSLinear=df_meggen.sumSunDayRef.interpolate(method='slinear'))
df_meggen = df_meggen.assign(InterpolationAkima=df_meggen.sumSunDayRef.interpolate(method='akima'))
df_meggen = df_meggen.assign(InterpolationPoly5=df_meggen.sumSunDayRef.interpolate(method='polynomial', order=5))
df_meggen = df_meggen.assign(InterpolationPoly7=df_meggen.sumSunDayRef.interpolate(method='polynomial', order=7))
df_meggen = df_meggen.assign(InterpolationSpline3=df_meggen.sumSunDayRef.interpolate(method='spline', order=3))
df_meggen = df_meggen.assign(InterpolationSpline4=df_meggen.sumSunDayRef.interpolate(method='spline', order=4))
df_meggen = df_meggen.assign(InterpolationSpline5=df_meggen.sumSunDayRef.interpolate(method='spline', order=5))
df_meggen.info()

Die Kontrolle mit der R-Squared Methode legt nahe, dass die Lineare-Methode am nächsten den realen Werten Nahe kommt. Aus diesem Grund wird mit dieser Methode die NA-Werte aufgefüllt. Bei lediglich 3 fehlenden Werten, stellt dies kein weiteres Problem dar.

In [ ]:
results = [(method, r2_score(df_meggen.sumSunDay, df_meggen[method])) for method in list(df_meggen)[3:]]
results_df = pd.DataFrame(np.array(results), columns=['Method', 'R_squared'])
results_df.sort_values(by='R_squared', ascending=False)

Beim Laden derselben Daten, werden die Daten nun mit der Linearen Interpolation aufgefüllt. Die Kontrolle zeigt, dass dies funktioniert hat. <br>
Als nächstes wird die Korrelation in Meggen betrachtet. Die Zeit-Spalte wird als Index verwendet. Mit einer Korrelation von -0.51 wird ein schwacher negativer Zusammenhang zwischen den Sonnenminuten und des Stromverbrauches bestätigt. Das bedeutet, je mehr Sonnenschein, desto tiefer der Stromverbrauch. Mehr Sonnenschein bedeuetet normalerweise auch höhere Temperaturen. Diese Analyse folgt in einem späteren Kapitel.<br>
Die Autocorrelation gibt anhand verschiedener Intervall an, wie stark die Werte von vorhergehenden Punkten in derselben Zeitreihe beeinflusst werden. Bei den Sonnenminuten gibt es keine regelmässigkeiten. Beim Strom sieht es anders aus. Mit jedem erweitern des Intervalls nimmt die Korrelation ab. Besonders bei 90-Tagen. Bei einem 1-Jahres Intervall ist die Korrelation wieder hoch. So kann zumindest angenommen werden, dass beim Strom eine wöchentliche und jährliche Korrelation besteht.


In [ ]:
df_meggen = df_sunPowerMeggen.copy()
df_meggen.drop(columns=['meteoParameter', 'meteoStation', 'bfsID', 'lawCityName'], inplace=True)
df_meggen['time'] = pd.to_datetime(df_meggen['time'])
df_meggen.sumSunDay.interpolate(method='linear', inplace=True)

#df_test = df_test.astype({'time': str})
df_meggen.index = df_meggen['time']
#df_test = df_test.astype({'sumSunDay': str})
df_meggen.drop(columns=['time'], inplace=True)
df_meggen.info()
df_meggen.head()

In [ ]:
print('Korrelation zwischen den Parametern:\n', df_meggen.corr())
print('\nAutocorrelation der täglichen Sonnenminuten und Stromverbrauch:')
autocorrOneSun = df_meggen['sumSunDay'].autocorr(lag=1)
print('Sonne 1-Tages-Intervall:\t', autocorrOneSun)
autocorrOneKwh = df_meggen['avgKwhConsum'].autocorr(lag=1)
print('Strom 1-Tages-Intervall:\t', autocorrOneKwh)
autocorr7Sun = df_meggen['sumSunDay'].autocorr(lag=7)
print('Sonne 7-Tages-Intervall:\t', autocorr7Sun)
autocorr7Kwh = df_meggen['avgKwhConsum'].autocorr(lag=7)
print('Strom 7-Tages-Intervall:\t', autocorr7Kwh)
autocorr30Sun = df_meggen['sumSunDay'].autocorr(lag=30)
print('Sonne 30-Tages-Intervall:\t', autocorr30Sun)
autocorr30Kwh = df_meggen['avgKwhConsum'].autocorr(lag=30)
print('Strom 30-Tages-Intervall:\t', autocorr30Kwh)

autocorr90Sun = df_meggen['sumSunDay'].autocorr(lag=90)
print('Sonne 90-Tages-Intervall:\t', autocorr90Sun)
autocorr90Kwh = df_meggen['avgKwhConsum'].autocorr(lag=90)
print('Strom 90-Tages-Intervall:\t', autocorr90Kwh)

autocorr365Sun = df_meggen['sumSunDay'].autocorr(lag=365)
print('Sonne 365-Tages-Intervall:\t', autocorr365Sun)
autocorr365Kwh = df_meggen['avgKwhConsum'].autocorr(lag=365)
print('Strom 365-Tages-Intervall:\t', autocorr365Kwh)

### Ausweitung der Analyse auf alle Gemeinden
Im nächsten Schritt wird die Korrelation-Analyse auf die weiteren Gemeinden ausgeweitet. Dazu wird die Analyse in West und Ost unterteilt, dies aufgrund der geografischen Standorte der Messstationen. Bei den westlichen Gemeinde werden die Daten bis zum 30.6.2024 verwendet.

In [ ]:
df_west = df_sunPower.query("meteoStation == 'SPF' | meteoStation == 'EGO'")
df_east = df_sunPower.query("meteoStation == 'LUZ' | meteoStation == 'MOA'")
df_west.info()
df_east.info()
df_west = df_west.copy()
df_west.head()
df_east = df_east.copy()
df_east.head()
df_west.drop(columns=['meteoParameter', 'meteoStation', 'lawCityName'], inplace=True)
df_west['time'] = pd.to_datetime(df_west['time'])
#df_west.dropna(inplace=True)
df_westPivot = df_west.pivot(index='time', columns=['bfsID'], values=['avgKwhConsum', 'sumSunDay'])
df_westPivot.info()

df_east.drop(columns=['meteoParameter', 'meteoStation', 'lawCityName'], inplace=True)
df_east['time'] = pd.to_datetime(df_east['time'])
#df_east.dropna(inplace=True)
df_eastPivot = df_east.pivot(index='time', columns='bfsID', values=['avgKwhConsum', 'sumSunDay'])
df_eastPivot.info()

In [ ]:
df_eastPivot.interpolate(method='linear', inplace=True)
df_eastPivot.info()
df_westPivot.interpolate(method='linear', inplace=True)
df_westPivot.info()

Für eine bessere optische Ansicht wird mit den Korrelation-Werten eine Heatmap erstellt. Zuerst für die östliche Seite. Was sofort auffällt sind die beiden dunklen Vierecke. Unten rechts sind die Korrelationen zwischen den Sonnenminuten zwischen den Gemeinden. Diese Werte sind sehr hoch. Somit scheint die Sonne vielerorts gleichzeitig und unterscheidet sich im Schnitt pro Tag nicht sehr stark. Oben links befinden sich die Korrelationen zwischen den Gemeinden und des Stromverbrauchs. Vielfach verhaltet sich der Stromverbrauch ähnlich zu den Gemeinden. Es gibt aber einige Ausreisser. Dies kann mit den aussergewöhnlichen Anomalien Zusammenhängen, welche bei einigen Ortschaften in der ersten Analyse erkannt wurden. Dies ist z.B. bei der Gemeinde Kriens (plz=6010/BFS-Nummer=1059). Die verbleibenden Quadranten geben die Korrelation zwischen dem Stromverbrauch und den Sonnenminuten an. Was sich bei Meggen angedeutet hat lässt sich mit der Heatmap bestätigen. Die Korrelation liegt im Bereich von ca. -0.5. Ausser bei denen, welche bereits eine schwache Korrelation bei den Zählerdaten vorweisen. Dort geht die Korrelation noch weiter zurück bis gegen 0.

In [ ]:
print('Korrelation zwischen den Parametern:\n', df_eastPivot.corr())

fig = plt.figure(figsize=(40, 40))
sns.heatmap(df_eastPivot.corr(), annot=True)
plt.show()

Die westliche Seite präsentiert sich in einem ähnlichen Gewand.

In [ ]:
print('Korrelation zwischen den Parametern:\n', df_eastPivot.corr())

fig = plt.figure(figsize=(40, 40))
sns.heatmap(df_westPivot.corr(), annot=True)
plt.show()

Um die Korrelationen innerhalb der Zeitreihe noch besser einschätzen zu können, wird ein Boxplot erstellt.

In [ ]:
columns = list(df_westPivot)
columns = list(df_eastPivot)

autocorrOneDay = []

#1 Day Correlation
for column in df_westPivot:
    autocorrOneSun = df_westPivot[column].autocorr(lag=1)
    col1 = column[0]+ 'OneDay'
    autocorrOneDay.append([col1,column[1],autocorrOneSun])

for column in df_eastPivot:
    autocorrOneSun = df_eastPivot[column].autocorr(lag=1)
    col1 = column[0]+ 'OneDay'
    autocorrOneDay.append([col1,column[1],autocorrOneSun])
#7 Day Correlation
for column in df_westPivot:
    autocorrOneSun = df_westPivot[column].autocorr(lag=7)
    col7 = column[0]+ '7Day'
    autocorrOneDay.append([col7,column[1],autocorrOneSun])

for column in df_eastPivot:
    autocorrOneSun = df_eastPivot[column].autocorr(lag=7)
    col7 = column[0]+ '7Day'
    autocorrOneDay.append([col7,column[1],autocorrOneSun])

#30 Day Correlation
for column in df_westPivot:
    autocorrOneSun = df_westPivot[column].autocorr(lag=30)
    col30 = column[0]+ '30Day'
    autocorrOneDay.append([col30,column[1],autocorrOneSun])

for column in df_eastPivot:
    autocorrOneSun = df_eastPivot[column].autocorr(lag=30)
    col30 = column[0]+ '30Day'
    autocorrOneDay.append([col30,column[1],autocorrOneSun])

for column in df_westPivot:
    autocorrOneSun = df_westPivot[column].autocorr(lag=90)
    col90 = column[0]+ '90Day'
    autocorrOneDay.append([col90,column[1],autocorrOneSun])

for column in df_eastPivot:
    autocorrOneSun = df_eastPivot[column].autocorr(lag=90)
    col90 = column[0]+ '90Day'
    autocorrOneDay.append([col90,column[1],autocorrOneSun])

for column in df_westPivot:
    autocorrOneSun = df_westPivot[column].autocorr(lag=365)
    col365 = column[0]+ '365Day'
    autocorrOneDay.append([col365,column[1],autocorrOneSun])

for column in df_eastPivot:
    autocorrOneSun = df_eastPivot[column].autocorr(lag=365)
    col365 = column[0]+ '365Day'
    autocorrOneDay.append([col365,column[1],autocorrOneSun])

df = pd.DataFrame(autocorrOneDay)
df = df.pivot(index=1, columns=0, values=2)
df.head()

Im Boxplot setzt sich die Erkenntnis von Meggen weiter. Das 90-Tag Intervall ist nicht relevant. Während die sonst gewählten Intervall eine höhere Korrelation vorweisen. So korreliert der Stromverbrauch bei 50% der Gemeinden mit dem Wert von 0.74 zum vor jährlichen Tagesschnitt. Im Bereich der Sonnenminuten gibt es keine bedeutende Korrelation.

In [ ]:
fig = plt.figure(figsize=(20, 10))
boxplot = sns.boxplot(df)
boxplot.set(title='Korrelation innerhalb der Zeitreihen mit verschiedenen Intervallen')
boxplot.tick_params(axis='x', labelrotation = 45)
plt.show()
df.describe()

Für den Boxplot der Korrelationen zwischen den Parametern werden normale Korrelationsmatrixen erstellt und die nicht notwendigen Index und Spalten entfernt.

In [ ]:
df_corrWest = df_westPivot[['avgKwhConsum', 'sumSunDay']].corr()
df_corrWest.drop(columns=['sumSunDay'], index=['avgKwhConsum'],inplace=True)
df_corrEast = df_eastPivot[['avgKwhConsum', 'sumSunDay']].corr()
df_corrEast.drop(columns=['sumSunDay'], index=['avgKwhConsum'],inplace=True)

Der Boxplot für den westlichen Teil zeigt das 75% der Korrelationswerte tiefer als ca. -0.3 sind, also eine stärkere Korrelation aufweisen. Wobei nur 25% der Korrelation zwischen ca. -0.475 und ca. -0.55 liegen. Der östliche Teil hat weniger Outlier und das 75% Quantil ein wenig früher bei ca. -0.29 startet. Generll gibt somit keine starke Verbindung zwischen diesen zwei Parametern.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16,10))
fig.suptitle('Vergleich der Korrelation zwischen Ost und West')
sns.boxplot(ax = axes[0], data=df_corrWest).set(title='Boxplot West')
sns.boxplot(ax = axes[1], data=df_corrEast).set(title='Boxplot Ost')
plt.show()
df_corrWest.describe()
df_corrEast.describe()

Abschliessend kann gesagt werden, dass die Sonne wohl keinen direkten Einfluss auf den Stromverbrauch hat.

## Mögliche Analyse Solarpanels, Stromverbrauch während Sonnenzeit und während Sonne weg ist
## Mögliche Analyse Zusammenhang des Zubaus von Solaranlagen und dem Trend des Rückgangs des Stromverbrauches über die restlichen drei Jahren.



## Datenanalyse Smartmeter mit Regendaten
Als nächster Schritt in der Analyse werden die Regendaten verwendet. In der Analyse wurde bereits festgestellt, dass die Daten nicht vollständig sind. In dieser Grafik ist gleich erkennbar, dass es da gewisse Ausschläge gibt, die nicht normal erscheinen. Für eine bessere Übersicht folgt später ein Facet Grid für jeden Standort. Die geladenen Daten beinhalten die aufsummierten Niederschläge in mm pro Tag und BFS-Nummer.

In [ ]:
df_rain = pd.read_sql_query("SELECT * FROM sumRainPerDayBfsId", connection)
df_rain[['dataTime', 'hour']] = df_rain['dataTime'].str.split(' ', expand=True)
df_rain['dataTime'] = pd.to_datetime(df_rain['dataTime'], format='%Y-%m-%d')
df_rain.drop(columns=['hour'], inplace=True)
df_rain.rename(columns={'dataTime':'time'}, inplace=True)
print(df_rain.count())
df_rain.head()

In [ ]:
plt.figure(figsize=(10, 6))
testPlot = sns.lineplot(x='time', y='rainSumDay', hue='meteoStation', data=df_rain).set(title='Summe des Regens pro Tag und Messstation', xlabel='Datum', ylabel='mm Regen')
plt.xticks(rotation=90)
plt.show()

Damit die fehlenden Daten später besser analysiert werden können, wird eine Pivot Tabelle erstellt.

In [ ]:
df_rainPivot = df_rain.pivot(index='time', columns=['bfsID'], values=['rainSumDay'])
df_rainPivot.info()

In [ ]:
df_power = pd.read_sql_query("SELECT * FROM dailySumSmartmeterBfsId", connection)
df_power['time'] = pd.to_datetime(df_power['time'])

Um die Vollständigkeit zu prüfen, wird eine Pivot Tabelle erstellt und mit der Info-Ansicht werden diese erkennbar. Die meisten Werte sind vorhanden oder es fehlt einer. Eine Aussnahme ist die BFS-Nummer 1100 und 1129 welche 30 bzw. 32 fehlende Daten aufweist.

In [ ]:
df_powerPivot = df_power.pivot(index='time', columns=['bfsID'], values=['avgKwhConsum'])
df_powerPivot.info()


Mit der Betrachtung der leeren Daten fällt auf, dass die Daten zu Beginn der Zeitreihe fehlen. Aus der Forecast Analyse wissen wir, dass sich der Stromverbrauch Saisonal verhält. Mit der Erkenntnis, dass sich der Stromverbrauch innerhalb des Monats stabil bewegt, wird die lineare Interpolation angewendet. Die Interpolation folgt zu einem späteren Zeitpunkt. Nachfolgend werden die Daten vorbereitet für die weitere Analyse.

In [ ]:
df_powerPivotNa = df_powerPivot[df_powerPivot.isna().any(axis=1)]
df_powerPivotNa.head(1000)

In [ ]:
df_rainPower = pd.merge(df_rain, df_power,on=['bfsID', 'time'])

In [ ]:
df_rainPower['time'] = pd.to_datetime(df_rainPower['time'])
df_rainPower.head()

In [ ]:
df_rainPower.drop(columns=['meteoParameter', 'lawCityName', 'meteoStation'], inplace=True)
df_rainPowerPivot = df_rainPower.pivot(index='time', columns=['bfsID'], values=['avgKwhConsum', 'rainSumDay'])
df_rainPowerPivot.head()
df_rainPowerPivot.info()

Wie aus dem untenstehenden Facet Grid zu entnehmen ist, gibt es viele nicht vorhandene Werte und dies auch über längere Zeiträume. Eine Interpolation über längere Zeiträume wird als nicht sinnvoll erachtet. Als Alternative werden die fehlende Zeiträume mit Daten von anderen Messstationen zum gleichen Zeitpunkt kopiert. Dieser Ansatz basiert auf der Idee, dass durch die relativ nahen geografischen Standorten, die Unterschiede nicht signifikant sein werden. Diese Annahme wird auch gestützt aufgrund der Korrelationsmatrix. Aus optischer Perspektive bewegt sich die Korrelation oft zwischen 0.7-0.8.

In [ ]:
# FacetGrid erstellen
g = sns.FacetGrid(df_rain, col='meteoStation', col_wrap=3, height=4, sharex=True, sharey=False)
g.map(sns.lineplot, 'time', 'rainSumDay')

# Achsenbeschriftungen und Titel hinzufügen
for ax in g.axes.flat:
    ax.set_xlabel('day')
    ax.set_ylabel('mm rain')

    # Intervall der x-Achsenbeschriftungen anpassen
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))

    # Rotation der Beschriftungen
    for label in ax.get_xticklabels():
        label.set_rotation(45)

plt.show()

In [ ]:
fig = plt.figure(figsize=(60, 60))
sns.heatmap(df_rainPowerPivot.corr(), annot=True)
plt.show()

Das kopieren der Werte wird solange fortgesetzt, bis keine na Werte mehr vorhanden sind. Am Ende wird ein Zähler ausgegeben, wie viele Iterationen vorgenommen wurden. 

In [ ]:
columns = df_rainPowerPivot['rainSumDay'].columns.tolist()
counter = 1
cycle = 0

while df_rainPowerPivot.isna().rainSumDay.any().sum() != 0:
    cycle += 1
    for column in columns:
        df_rainPowerPivot[('rainSumDay', column)].fillna(df_rainPowerPivot[('rainSumDay', columns[counter])], inplace=True)
        if counter == len(columns)-1:
            counter = 0
        else:
            counter += 1
print("Anzahl durchgeführter Zyklen zum kopieren:\t", cycle)
df_rainPowerPivot.info()
df_rainPowerPivot.head()


Nun werden die Stromdaten mit einer linearen Interpolation ergänzt. Die Kontrolle ergibt, dass die Zeitreihe von 1129 nicht aufgefüllt wird. Dies liegt wohl daran, dass ein Anfangswert fehlt. Aus diesem Grund werden die restlichen Werte mit einem Durchschnittswert der entsprechenden Spalte aufgefüllt.

In [ ]:
df_rainPowerPivot.avgKwhConsum.interpolate(method='linear', inplace=True)
df_powerPivotNa = df_rainPowerPivot[df_rainPowerPivot.isna().any(axis=1)]
df_powerPivotNa.head()

In [ ]:
df_rainPowerPivot[('avgKwhConsum', 1129)].fillna(df_rainPowerPivot[('avgKwhConsum', 1129)].mean(), inplace=True)
df_powerPivotNa = df_rainPowerPivot[df_rainPowerPivot.isna().any(axis=1)]
print('Keine fehlende Werte?:\t', df_rainPowerPivot.isna().any().sum() == 0)
df_powerPivotNa.head()

Nachdem die Daten kopiert wurden, wird die Tabelle wieder umgestellt, damit die Grafiken erstellt werden können.

In [ ]:
df_rainPowerNew = pd.DataFrame(columns=['time', 'rainSumDay', 'bfsID', 'avgKwhConsum'])
df_rainNew = df_rainPowerPivot['rainSumDay']
df_powerNew = df_rainPowerPivot['avgKwhConsum']
df_rainNew = df_rainNew.unstack().reset_index()
df_rainNew.rename(columns={0:'rainSumDay'}, inplace=True)
df_powerNew = df_powerNew.unstack().reset_index()
df_powerNew.rename(columns={0:'avgKwhConsum'}, inplace=True)

df_rainNew.head()
df_powerNew.head()
df_rainPowerNew = pd.merge(df_rainNew, df_powerNew, on=['bfsID', 'time'])
df_rainPowerNew.head()

In [ ]:
df_rainPowerNew.info()

Zur Kontrolle werden die Daten erneut in einem Facet Grid dargestellt. Die Daten sehen nun vollständig aus. Bei der BFS-Nummer 1024 gibt es noch einen langen Bereich welcher keine Daten anzeigt aber nicht als NAN gewertet wurden. Da dies nicht weiter verfiziert werden kann, wird dieser Datenstamm so belassen. Dasselbe gilt für die BFS-Nummern 1121 und 1143. Diese Daten sehen so aus, als gäbe es einen aussergewöhnlichen Zeitraum, doch es fehlen Anhaltspunkte diese weiter zu untersuchen. <br>
Mit diesen Datenreihen können nun weitere Untersuchungen angestellt werden.

In [ ]:
# FacetGrid erstellen
g = sns.FacetGrid(df_rainPowerNew, col='bfsID', col_wrap=3, height=4, sharex=True, sharey=False)
g.map(sns.lineplot, 'time', 'rainSumDay')

# Achsenbeschriftungen und Titel hinzufügen
for ax in g.axes.flat:
    ax.set_xlabel('day')
    ax.set_ylabel('mm rain')

    # Intervall der x-Achsenbeschriftungen anpassen
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))

    # Rotation der Beschriftungen
    for label in ax.get_xticklabels():
        label.set_rotation(45)

plt.show()

### Erstanalyse am Beispiel der Gemeinde Meggen
Wie bei der Analyse mit den Smartmeterdaten und Sonnenminuten wird zunächst mit einem reduzierten Dataset gearbeitet von der Gemeinde Meggen.

In [ ]:
df_1063 = df_rainPowerNew.query('bfsID == 1063')
df_1063.info()

Die Zeitreihe wird erneut kontrolliert und scheint vollständig zu sein.

In [ ]:
df_1063.drop(columns=['bfsID'], inplace=True)
df_1063.info()
print(pd.date_range(start = '2020-12-31', end='2024-03-06').difference(df_1063['time']))

Der Stromverbrauch wurde bereits analysiert. Der Niederschlag zeigt ebenfalls eine leichte Saisonalität. In den Bereichen Sommer und Herbst scheint es tendenziell mehr Niederschlag zu geben, während es im Winter nach weniger Niederschlag aussieht.

In [ ]:
meggenPlot, ax1 = plt.subplots(figsize=(10, 6))
sns.lineplot(x='time', y='avgKwhConsum', data=df_1063, ax=ax1).set(title='Vergleich des durchschnittlichen Stromverbrauches zu dem Niederschlag', xlabel='Datum', ylabel='Verbrauch in kWh')
ax2 = ax1.twinx()
sns.lineplot(data=df_1063, x='time', y='rainSumDay', ax=ax2, color='red').set(ylabel='Niederschlag mm')
plt.xticks(rotation=90)
plt.show()

Als nächstes wird die Korrelation in Meggen betrachtet. Die Zeit-Spalte wird als Index verwendet. Mit einer Korrelation von -0.017 besteht kein Zusammenhang zwischen dem Niederschlag und des Stromverbrauches.<br>
Die Autocorrelation gibt anhand verschiedener Intervall an, wie stark die Werte von vorhergehenden Punkten in derselben Zeitreihe beeinflusst werden. Bei dem Niederschlag gibt es keine Regelmässigkeiten. Die Korrelationen bezüglich dem Strom wurden bereits in der vorhergehenden Analyse mit der Sonne beleuchtet.

In [ ]:
df_1063.set_index('time', inplace=True)
print('Korrelation zwischen den Parametern:\n', df_1063.corr())
print('\nAutocorrelation des täglichen Niederschlages und Stromverbrauch:')
autocorrOne = df_1063['rainSumDay'].autocorr(lag=1)
print('Niederschlag 1-Tages-Intervall:\t', autocorrOne)
autocorrOneKwh = df_1063['avgKwhConsum'].autocorr(lag=1)
print('Strom 1-Tages-Intervall:\t', autocorrOneKwh)
autocorr7 = df_1063['rainSumDay'].autocorr(lag=7)
print('Niederschlag 7-Tages-Intervall:\t', autocorr7)
autocorr7Kwh = df_1063['avgKwhConsum'].autocorr(lag=7)
print('Strom 7-Tages-Intervall:\t', autocorr7Kwh)
autocorr30 = df_1063['rainSumDay'].autocorr(lag=30)
print('Niederschlag 30-Tages-Intervall:\t', autocorr30)
autocorr30Kwh = df_1063['avgKwhConsum'].autocorr(lag=30)
print('Strom 30-Tages-Intervall:\t', autocorr30Kwh)

autocorr90 = df_1063['rainSumDay'].autocorr(lag=90)
print('Niederschlag 90-Tages-Intervall:\t', autocorr90)
autocorr90Kwh = df_1063['avgKwhConsum'].autocorr(lag=90)
print('Strom 90-Tages-Intervall:\t', autocorr90Kwh)

autocorr365 = df_1063['rainSumDay'].autocorr(lag=365)
print('Niederschlag 365-Tages-Intervall:\t', autocorr365)
autocorr365Kwh = df_1063['avgKwhConsum'].autocorr(lag=365)
print('Strom 365-Tages-Intervall:\t', autocorr365Kwh)

### Ausweitung der Analyse auf alle Gemeinden
Im nächsten Schritt wird die Korrelation-Analyse auf die weiteren Gemeinden ausgeweitet. Hier wird keine Auftrennung in Ost und West benötigt, da die fehlenden Werte aufgefüllt wurden.

In [ ]:
df_rainPowerPivot.info()

Für eine bessere optische Ansicht wird mit den Korrelation-Werten eine Heatmap erstellt. Was sofort auffällt sind die beiden violetten Vierecke die herausstechen. Das sind die Korrelationswerte zwischen dem Niederschlag und des Stromverbrauches pro Gemeinde. Aufgrund der Farb-Skallierung erkennt man den erkannten Trend bei der Meggen-Analyse. Die Korrelationen sind sehr schwach, daher besteht wohl eher kein Zusammenhang zwischen dem Niederschlag und dem Stromverbrauch. Unten rechts sind die Korrelationen zwischen dem Niederschlag zwischen den Gemeinden. Diese Werte sind hoch und deuten somit an, dass sich der Niederschlag im Kanton Luzern ähnlich verhält. Oben links befinden sich die Korrelationen zwischen den Gemeinden und des Stromverbrauchs. Vielfach verhaltet sich der Stromverbrauch ähnlich zu den Gemeinden. Es gibt aber einige Ausreisser. Dies kann mit den aussergewöhnlichen Anomalien Zusammenhängen, welche bei einigen Ortschaften in der ersten Analyse erkannt wurden. Dies ist z.B. bei der Gemeinde Kriens (plz=6010/BFS-Nummer=1059).

In [ ]:
print('Korrelation zwischen den Parametern:\n', df_rainPowerPivot.corr())

fig = plt.figure(figsize=(60, 60))
sns.heatmap(df_rainPowerPivot.corr(), annot=True)
plt.show()

Um die Korrelationen der Zeitreihen noch besser zu verstehen folgt der Test, der Autokorrelation.

In [ ]:
columns = list(df_rainPowerPivot)

autocorrOneDay = []

#1 Day Correlation
for column in df_rainPowerPivot:
    autocorrOneSun = df_rainPowerPivot[column].autocorr(lag=1)
    col1 = column[0]+ 'OneDay'
    autocorrOneDay.append([col1,column[1],autocorrOneSun])

#7 Day Correlation
for column in df_rainPowerPivot:
    autocorrOneSun = df_rainPowerPivot[column].autocorr(lag=7)
    col7 = column[0]+ '7Day'
    autocorrOneDay.append([col7,column[1],autocorrOneSun])

#30 Day Correlation
for column in df_rainPowerPivot:
    autocorrOneSun = df_rainPowerPivot[column].autocorr(lag=30)
    col30 = column[0]+ '30Day'
    autocorrOneDay.append([col30,column[1],autocorrOneSun])

for column in df_rainPowerPivot:
    autocorrOneSun = df_rainPowerPivot[column].autocorr(lag=90)
    col90 = column[0]+ '90Day'
    autocorrOneDay.append([col90,column[1],autocorrOneSun])

for column in df_rainPowerPivot:
    autocorrOneSun = df_rainPowerPivot[column].autocorr(lag=365)
    col365 = column[0]+ '365Day'
    autocorrOneDay.append([col365,column[1],autocorrOneSun])

df = pd.DataFrame(autocorrOneDay)
df = df.pivot(index=1, columns=0, values=2)
df.head()

Im Boxplot setzt sich die Erkenntnis von Meggen weiter. Die Korrelationen innerhalb der Intervalle liegen praktisch bei 0 und zeigen somit keinen Zusammenhang. Dies bedeutet z.B., wenn es heute regnet, muss dies nicht zwingend bedeuten, dass es dies in einem Jahr ebenfalls tut. Das Wetter in einem Jahr kann nicht durch das heutige Wetter vorhergesagt werden. Beim Strom kann dies eher behauptet werden.

In [ ]:
fig = plt.figure(figsize=(20, 10))
sns.boxplot(df).set(title='Korrelation innerhalb der Zeitreihen mit verschiedenen Intervallen')
plt.show()
df.describe()

Für den Boxplot der Korrelationen zwischen den Parametern werden normale Korrelationsmatrixen erstellt und die nicht notwendigen Index und Spalten entfernt.

In [ ]:
df_corrRain = df_rainPowerPivot[['avgKwhConsum', 'rainSumDay']].corr()
df_corrRain.drop(columns=['rainSumDay'], index=['avgKwhConsum'],inplace=True)

Der Boxplot das 75% der Korrelationswerte zwischen ca. 0.08 und -0.15 sind, also keine Korrelation aufweisen. Die restlichen 25% verteilen sich als Outlier oben und unten, wobei der Höchstwert bei ca. 0.2 liegt und der tiefste Wert bei -0.2.

In [ ]:
fig = plt.figure(figsize=(16,10))
sns.boxplot(data=df_corrRain).set(title='Boxplot über die Korrelationen zwischen dem Niederschlag und Stromverbrauch')
plt.show()
df_corrRain.describe()

Abschliessend kann festgehalten werden, dass der Niederschlag an sich keine Auswirkung auf den Stromverbrauch hat.

## Datenanalyse Smartmeter mit der Temperatur
Zuerst werden die Daten aus der Datenbank geladen und gleich im korrekten Zeitformat gesichert. Aus der Analyse ist bekannt, dass gewisse Stationen fehlende Daten ausweisen. Aus bestätigt sich erneut mit den fehlenden Temperatur Daten. Für die Analyse werden werden die durchschnittlichen Temperaturwerte pro Tag und BFS-Nummer verwendet.

In [ ]:
df_temp = pd.read_sql_query("SELECT * FROM avgTempPerDayBfsId", connection)
df_temp[['dataTime', 'hour']] = df_temp['dataTime'].str.split(' ', expand=True)
df_temp['dataTime'] = pd.to_datetime(df_temp['dataTime'], format='%Y-%m-%d')
df_temp.drop(columns=['hour'], inplace=True)
df_temp.rename(columns={'dataTime':'time'}, inplace=True)
print(df_temp.count())
df_temp.head()

Um eine eine erste Übersicht zu erhalten, werden die Daten in einer Grafik dargestellt. Man erkennt sogleich die naheliegende starke Saisonalität. Im Winter ist es kälter und im Sommer wärmer.

In [ ]:
plt.figure(figsize=(10, 6))
testPlot = sns.lineplot(x='time', y='avgTempDay', hue='meteoStation', data=df_temp).set(title='Durchschnittliche Temperatur pro Tag', xlabel='Datum', ylabel='Temperatur Grad Celsius')
plt.xticks(rotation=90)
plt.show()

Um festzustellen, wo sich die fehlenden Daten befinden, wird eine Pivot Tabelle erstellt. Die fehlenden Werte sind mehrfach verteilt und müssen zu einem späteren Zeitpunkt einer Interpollation oder einer anderen Methode aufgefüllt werden.

In [ ]:
df_tempPivot = df_temp.pivot(index='time', columns=['bfsID'], values=['avgTempDay'])
df_rainPivot.info()

Für den Stromverbrauch, werden die täglich aufsummierten kWh pro BFS-Nummer geladen. Die Tabelle wird pivotiert um die NAN-Werte zu identifizieren. Da es sich um dieselben Stromverbrauch-Daten handelt, wie in den vorhergehenden Analysen, werden diese zu einem späteren Zeitpunkt gleich behandelt bezüglich den NAN-Werten.

In [ ]:
df_power = pd.read_sql_query("SELECT * FROM dailySumSmartmeterBfsId", connection)
df_power['time'] = pd.to_datetime(df_power['time'])

In [ ]:
df_powerPivot = df_power.pivot(index='time', columns=['bfsID'], values=['avgKwhConsum'])
df_powerPivot.info()

Für die spätere Behandlung der NAN-Werte wird eine kombinierte Pivot Tabelle mit den Stromverbrauchs- und Temperatur-Daten gebildet.

In [ ]:
df_tempPower = pd.merge(df_temp, df_power,on=['bfsID', 'time'])
df_tempPower['time'] = pd.to_datetime(df_tempPower['time'])
df_tempPower.head()

In [ ]:
df_tempPower.drop(columns=['meteoParameter', 'lawCityName', 'meteoStation'], inplace=True)
df_tempPowerPivot = df_tempPower.pivot(index='time', columns=['bfsID'], values=['avgKwhConsum', 'avgTempDay'])
df_rainPowerPivot.head()
df_rainPowerPivot.info()

Anhand der Grafiken, lassen sich nicht lange Zeiträume feststellen, wo es fehlende Daten gibt. Aus diesem Grund wird eine lineare Interpolation angewendet. Als Alternative hätten die fehlende Zeiträume mit Daten von anderen Messstationen zum gleichen Zeitpunkt kopiert werden können. Dieser Ansatz basiert auf der Idee, dass durch die relativ nahen geografischen Standorten, die Unterschiede nicht signifikant sein werden. Diese Annahme wird auch gestützt aufgrund der Korrelationsmatrix. Aus optischer Perspektive bewegen sich die Korrelationen um den Wert von 0.9.

In [ ]:
# FacetGrid erstellen
g = sns.FacetGrid(df_temp, col='meteoStation', col_wrap=3, height=4, sharex=True, sharey=False)
g.map(sns.lineplot, 'time', 'avgTempDay')

# Achsenbeschriftungen und Titel hinzufügen
for ax in g.axes.flat:
    ax.set_xlabel('day')
    ax.set_ylabel('temperature celsius')

    # Intervall der x-Achsenbeschriftungen anpassen
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))

    # Rotation der Beschriftungen
    for label in ax.get_xticklabels():
        label.set_rotation(45)

plt.show()

In [ ]:
fig = plt.figure(figsize=(60, 60))
sns.heatmap(df_tempPowerPivot.corr(), annot=True)
plt.show()

Zuerst wird die Interpolation der Temperaturen vorgenommen.

In [ ]:
df_tempPowerPivot.avgTempDay.interpolate(method='linear', inplace=True)

Nun folgen die Interpolationen der Stromverbrauchsdaten nach dem bereits bekannten Muster.

In [ ]:
df_tempPowerPivot.avgKwhConsum.interpolate(method='linear', inplace=True)
df_tempPivotNa = df_tempPowerPivot[df_tempPowerPivot.isna().any(axis=1)]
df_tempPivotNa.head()

In [ ]:
df_tempPowerPivot[('avgKwhConsum', 1129)].fillna(df_tempPowerPivot[('avgKwhConsum', 1129)].mean(), inplace=True)
df_powerPivotNa = df_tempPowerPivot[df_tempPowerPivot.isna().any(axis=1)]
print('Keine fehlende Werte?:\t', df_tempPowerPivot.isna().any().sum() == 0)
df_tempPivotNa.head(1000)

Die Schlusskontrolle zeigt, dass bei der BFS-Nummer 1129 der Start der Temperatur Zeitreihe fehlen. Aus diesem Grund werden diese Daten von einer anderen Station kopiert und ergänzt.

In [ ]:
columns = df_tempPowerPivot['avgTempDay'].columns.tolist()
counter = 1
cycle = 0

while df_tempPowerPivot.isna().avgTempDay.any().sum() != 0:
    cycle += 1
    for column in columns:
        df_tempPowerPivot[('avgTempDay', column)].fillna(df_tempPowerPivot[('avgTempDay', columns[counter])], inplace=True)
        if counter == len(columns)-1:
            counter = 0
        else:
            counter += 1
print("Anzahl durchgeführter Zyklen zum kopieren:\t", cycle)
df_rainPowerPivot.info()
df_rainPowerPivot.head()

Ein Kopierzyklus wurde durchgeführt. Die Abschlusskontrolle findet keine weiteren NAN-Werte aus diesem Grund kann nun mit der weiteren Analyse fortgefahren werden.

In [ ]:
print('Keine fehlende Werte?:\t', df_tempPowerPivot.isna().any().sum() == 0)


Die vollständigen Zeitreihen werden wieder entpivotiert für die weitere Verwendung.

In [ ]:
df_tempPowerNew = pd.DataFrame(columns=['time', 'avgTempDay', 'bfsID', 'avgKwhConsum'])
df_tempNew = df_tempPowerPivot['avgTempDay']
df_powerNew = df_tempPowerPivot['avgKwhConsum']
df_tempNew = df_tempNew.unstack().reset_index()
df_tempNew.rename(columns={0:'avgTempDay'}, inplace=True)
df_powerNew = df_powerNew.unstack().reset_index()
df_powerNew.rename(columns={0:'avgKwhConsum'}, inplace=True)

df_tempNew.head()
df_tempNew.head()
df_tempPowerNew = pd.merge(df_tempNew, df_powerNew, on=['bfsID', 'time'])
df_tempPowerNew.head()

Zur Kontrolle werden die Daten erneut in einem Facet Grid dargestellt. Es bestehen noch immer keine Auffälligkeiten. Aus diesem Grund werden die weiteren Analysen mit diesen Zeitreihen durchgeführt.

In [ ]:
df_rainPowerNew.info()

In [ ]:
# FacetGrid erstellen
g = sns.FacetGrid(df_tempPowerNew, col='bfsID', col_wrap=3, height=4, sharex=True, sharey=False)
g.map(sns.lineplot, 'time', 'avgTempDay')

# Achsenbeschriftungen und Titel hinzufügen
for ax in g.axes.flat:
    ax.set_xlabel('day')
    ax.set_ylabel('mm rain')

    # Intervall der x-Achsenbeschriftungen anpassen
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))

    # Rotation der Beschriftungen
    for label in ax.get_xticklabels():
        label.set_rotation(45)

plt.show()

### Erstanalyse am Beispiel der Gemeinde Meggen
Wie bei den vorhergehenden Analysen mit den wird zunächst mit einem reduzierten Dataset gearbeitet von der Gemeinde Meggen.

In [ ]:
df_1063 = df_tempPowerNew.query('bfsID == 1063')
df_1063.info()

Die Zeitreihe wird erneut kontrolliert und ist vollständig.

In [ ]:
df_1063.drop(columns=['bfsID'], inplace=True)
df_1063.info()
print(pd.date_range(start = '2020-12-31', end='2024-03-06').difference(df_1063['time']))

Wie der Stromverbrauch, zeigt die Temperatur ebenfalls eine starke Saisonalität. Zudem scheint diese, sehr gegengleich auszuschlagen. Es zeichnet sich eine gute Korrelation ab.

In [ ]:
meggenPlot, ax1 = plt.subplots(figsize=(10, 6))
sns.lineplot(x='time', y='avgKwhConsum', data=df_1063, ax=ax1).set(title='Vergleich des durchschnittlichen Stromverbrauches zu der durchschnittlichen Temperatur', xlabel='Datum', ylabel='Verbrauch in kWh')
ax2 = ax1.twinx()
sns.lineplot(data=df_1063, x='time', y='avgTempDay', ax=ax2, color='red').set(ylabel='Temperatur Grad Celsius')
plt.xticks(rotation=90)
plt.show()

Als nächstes wird die Korrelation in Meggen betrachtet. Die Zeit-Spalte wird als Index verwendet. Mit einer Korrelation von -0.87 besteht ein Zusammenhang zwischen der Temperatur und des Stromverbrauches. Durch das negative Vorzeichen, bedeutet dies ein umgekehrter Zusammenhang. Je tiefer die Temperatur, desto höher ist der Stromverbrauch.<br>
Die Autocorrelation gibt anhand verschiedener Intervall an, wie stark die Werte von vorhergehenden Punkten in derselben Zeitreihe beeinflusst werden. Hier gibt es in beiden Fällen eine starke Korrelation auf den entsprechenden Intervallen. Ausgenommen das vierteljährliche Intervall. Die Temperatur ist im Folgejahr in einem ähnlichen Bereich wie heute.

In [ ]:
df_1063.set_index('time', inplace=True)
print('Korrelation zwischen den Parametern:\n', df_1063.corr())
print('\nAutocorrelation der durchschnittlichen Temperatur und Stromverbrauch:')
autocorrOne = df_1063['avgTempDay'].autocorr(lag=1)
print('Temperatur 1-Tages-Intervall:\t', autocorrOne)
autocorrOneKwh = df_1063['avgKwhConsum'].autocorr(lag=1)
print('Strom 1-Tages-Intervall:\t', autocorrOneKwh)
autocorr7 = df_1063['avgTempDay'].autocorr(lag=7)
print('Temperatur 7-Tages-Intervall:\t', autocorr7)
autocorr7Kwh = df_1063['avgKwhConsum'].autocorr(lag=7)
print('Strom 7-Tages-Intervall:\t', autocorr7Kwh)
autocorr30 = df_1063['avgTempDay'].autocorr(lag=30)
print('Temperatur 30-Tages-Intervall:\t', autocorr30)
autocorr30Kwh = df_1063['avgKwhConsum'].autocorr(lag=30)
print('Strom 30-Tages-Intervall:\t', autocorr30Kwh)

autocorr90 = df_1063['avgTempDay'].autocorr(lag=90)
print('Temperatur 90-Tages-Intervall:\t', autocorr90)
autocorr90Kwh = df_1063['avgKwhConsum'].autocorr(lag=90)
print('Strom 90-Tages-Intervall:\t', autocorr90Kwh)

autocorr365 = df_1063['avgTempDay'].autocorr(lag=365)
print('Temperatur 365-Tages-Intervall:\t', autocorr365)
autocorr365Kwh = df_1063['avgKwhConsum'].autocorr(lag=365)
print('Strom 365-Tages-Intervall:\t', autocorr365Kwh)

### Ausweitung der Analyse auf alle Gemeinden
Im nächsten Schritt wird die Korrelation-Analyse auf die weiteren Gemeinden ausgeweitet. Hier wird keine Auftrennung in Ost und West benötigt, da die fehlenden Werte aufgefüllt wurden.

In [ ]:
df_tempPowerPivot.info()

Für eine bessere optische Ansicht wird mit den Korrelation-Werten eine Heatmap erstellt. Was sofort auffällt sind die beiden violetten Vierecke die herausstechen. Das sind die Korrelationswerte zwischen der Temperatur und des Stromverbrauches pro Gemeinde. Aufgrund der Farb-Skallierung erkennt man den erkannten Trend bei der Meggen-Analyse. Die Korrelationen sind eher auf der starken negativen Seite, daher besteht wohl Zusammenhang zwischen dem Niederschlag und dem Stromverbrauch. Unten rechts sind die Korrelationen zwischen den Temperaturen zwischen den Gemeinden. Diese Werte sind hoch und deuten somit an, dass sich die Temperaturen im Kanton Luzern sehr ähnlich verhalten. Oben links befinden sich die Korrelationen zwischen den Gemeinden und des Stromverbrauchs. Vielfach verhaltet sich der Stromverbrauch ähnlich zu den Gemeinden. Es gibt aber einige Ausreisser. Dies kann mit den aussergewöhnlichen Anomalien Zusammenhängen, welche bei einigen Ortschaften in der ersten Analyse erkannt wurden. Dies ist z.B. bei der Gemeinde Kriens (plz=6010/BFS-Nummer=1059).

In [ ]:
print('Korrelation zwischen den Parametern:\n', df_tempPowerPivot.corr())

fig = plt.figure(figsize=(60, 60))
sns.heatmap(df_tempPowerPivot.corr(), annot=True)
plt.show()

Die Korrelationen werden nun noch weiter betrachtet zunächst in der Autokorrelation.

In [ ]:
columns = list(df_tempPowerPivot)

autocorrOneDay = []

#1 Day Correlation
for column in df_tempPowerPivot:
    autocorrOneSun = df_tempPowerPivot[column].autocorr(lag=1)
    col1 = column[0]+ 'OneDay'
    autocorrOneDay.append([col1,column[1],autocorrOneSun])

#7 Day Correlation
for column in df_tempPowerPivot:
    autocorrOneSun = df_tempPowerPivot[column].autocorr(lag=7)
    col7 = column[0]+ '7Day'
    autocorrOneDay.append([col7,column[1],autocorrOneSun])

#30 Day Correlation
for column in df_tempPowerPivot:
    autocorrOneSun = df_tempPowerPivot[column].autocorr(lag=30)
    col30 = column[0]+ '30Day'
    autocorrOneDay.append([col30,column[1],autocorrOneSun])

for column in df_tempPowerPivot:
    autocorrOneSun = df_tempPowerPivot[column].autocorr(lag=90)
    col90 = column[0]+ '90Day'
    autocorrOneDay.append([col90,column[1],autocorrOneSun])

for column in df_tempPowerPivot:
    autocorrOneSun = df_tempPowerPivot[column].autocorr(lag=365)
    col365 = column[0]+ '365Day'
    autocorrOneDay.append([col365,column[1],autocorrOneSun])

df = pd.DataFrame(autocorrOneDay)
df = df.pivot(index=1, columns=0, values=2)
df.head()

Auf die Boxplots des Stromverbrauchs wird nicht weiter eingegangen. Die Betrachtung der Boxplots über die Temperatur zeigen einen sehr engen Zusammenhang und keine grosse Streuung der Werte. Das 90-Tag-Intervall ist wie dem Stromverbrauch irrelevant.

In [ ]:
fig = plt.figure(figsize=(20, 10))
sns.boxplot(df).set(title='Korrelation innerhalb der Zeitreihen mit verschiedenen Intervallen')
plt.show()
df.describe()

Für die Boxplots der Korrelationen zwischen den Temperaturen und Stromverbrauch, werden die nicht benötigten Daten entfernt.

In [ ]:
df_corrTemp = df_tempPowerPivot[['avgKwhConsum', 'avgTempDay']].corr()
df_corrTemp.drop(columns=['avgTempDay'], index=['avgKwhConsum'],inplace=True)

Mit einem Median bei ca. -0.85 kann von einem starken Zusammenhang ausgegangen werden zwischen der Temperatur und dem Stromverbrauch. Auch bei der Betrachtung des 75% Quantil liegen alle Werte unter ca. -0.65. Dies deutet weiterhin auf einen Zusammenhang hin. Da es wahrscheinlich fehlerbehaftete Smartmeterdaten gibt, könnte die Korrelation noch höher sein.

In [ ]:
fig = plt.figure(figsize=(16,10))
sns.boxplot(data=df_corrTemp).set(title='Boxplot über die Korrelationen zwischen den Temperaturen und Stromverbrauch')
plt.show()
df_corrTemp.describe()

Abschliessend kann gesagt werden, dass die Temperatur und der Stromverbrauch zusammenhängen.

# Forecasting
In diesem Kapitel wird versucht ein Vorhersage-Modell zu erstellen basierend auf den vorliegenden Erkenntnissen. Zuerst wird versucht anhand der Stromverbrauchs-Zeitreihe von Meggen eine Vorhersage zu treffen. Danach folgt ein Versuch mittels eines Regressions-Modell den Stromverbrauch mit der Temperatur vorherzusagen.

## Vorhersage Stromverbrauch
Für die Vorhersage des Stromverbrauchs wird erneut die Zeitreihe von Meggen verwendet mit den täglichen durchschnittlichen Stromverbrauchsdaten.

In [ ]:
df_powerMeggen = df_1063.copy()
df_powerMeggen.drop(columns=['avgTempDay'], inplace=True)

In [ ]:
df_powerMeggen.info()

Um die Daten zu glätten werden verschiedene Methoden versucht anzuwenden. Darunter gehören die:
- Rolling Mean Intervall 7
- Rolling Mean Intervall 365
- Rolling Standard Deviation Intervall 7
- Rolling Standard Deviation Intervall 365
Die Intervalle wurden ausgewählt aufgrund der Autocorrelation Analyse in dem vorherigen Kapitel.

In [ ]:
rolmean7 = df_powerMeggen.rolling(window=7).mean()
rolmean14 = df_powerMeggen.rolling(window=14).mean()
rolmean30 = df_powerMeggen.rolling(window=30).mean()
rolmean90 = df_powerMeggen.rolling(window=90).mean()
rolmean365 = df_powerMeggen.rolling(window=365).mean()
rolstd7 = df_powerMeggen.rolling(window=7).std()
rolstd14 = df_powerMeggen.rolling(window=14).std()
rolstd30 = df_powerMeggen.rolling(window=30).std()
rolstd90 = df_powerMeggen.rolling(window=90).std()
rolstd365 = df_powerMeggen.rolling(window=365).std()

Rot wird die originale Zeitreihe dargestellt. Die anderen Farben repräsentieren die Versuche die Kurve zu glätten. Mit dem wöchigen Intervall werden die Daten durch den Rolling Mean gut dargestellt. Mit der Standardabweichung wird das Rauschen entlang des Durchschnittes dargestellt. Bei der Betrachtung des Intervalls von 365 wird die Saisonalität herausgefiltert und es zeichnet sich ein leichter negativer Trend ab. Die Saisonalität muss noch weiter untersucht werden. <br>
Mit dem Rolling Mean können die Kurven geglättet werden, die Kurve mit dem einem Intervall von sieben, beschreibt die Daten sehr gut und behält die Spitzen. Das 30-Tage Intervall glättet die Kurven am Besten ohne die jährliche Saisonalität zu verlieren. Das 90-Tage Intervall glättet noch besser, ist aber bereits stark verschoben. <br>
Die Standardabweichung wird immer glatter aber höher, je höher das Intervall gewählt wird.

In [ ]:
fig = plt.figure()
fig = plt.figure(figsize=(20, 6))
sns.lineplot(data=df_powerMeggen, x='time', y='avgKwhConsum', label='Täglicher durchschnittler Stromverbrauch pro Smartmeter', color='red').set(title='Vergleich täglicher Stromverbrauch für Meggen', xlabel='Zeit', ylabel='Täglicher durchschnittler Verbrauch pro Smartmeter (kWh)')
sns.lineplot(x='time', y='avgKwhConsum', data=rolmean7, label='7-Rolling Mean Strom kWh', color='blue').set()
sns.lineplot(x='time', y='avgKwhConsum', data=rolstd7, label='7-Rolling Standard Deviation kWh', color='green').set()
sns.lineplot(x='time', y='avgKwhConsum', data=rolmean14, label='14-Rolling Mean Strom kWh', color='brown').set()
sns.lineplot(x='time', y='avgKwhConsum', data=rolstd14, label='14-Rolling Standard Deviation kWh', color='navy').set()
sns.lineplot(x='time', y='avgKwhConsum', data=rolmean30, label='30-Rolling Mean Strom kWh', color='orange').set()
sns.lineplot(x='time', y='avgKwhConsum', data=rolstd30, label='30-Rolling Standard Deviation kWh', color='grey').set()
sns.lineplot(x='time', y='avgKwhConsum', data=rolmean90, label='90-Rolling Mean Strom kWh', color='yellow').set()
sns.lineplot(x='time', y='avgKwhConsum', data=rolstd90, label='90-Rolling Standard Deviation kWh', color='purple').set()
sns.lineplot(x='time', y='avgKwhConsum', data=rolmean365, label='365-Rolling Mean Strom kWh', color='black').set()
sns.lineplot(x='time', y='avgKwhConsum', data=rolstd365, label='365-Rolling Standard Deviation kWh', color='grey').set()
plt.show()

### Decomposition
Die Zeitreihe wird nun zerlegt in verschiedene Perioden um verschiedene Saisonalitäten festzustellen.<br>
Anhand des Trends ist erkennbar, dass der Stromverbrauch über die letzten drei Jahre zurückgegangen ist. Innerhalb der jeder Periode sind Saisonalitäten erkennbar.

In [ ]:
ax = plt.figure(figsize = (30,80))
mstl = MSTL(df_powerMeggen, periods=[7, 14, 30, 365])
res = mstl.fit()
ax = res.plot()
plt.xticks(rotation=90)
plt.show()

Die gleiche Analyse mit dem Rolling Mean über 30 Tage zeigt noch immer die Saisonalität zu den gegebenen Perioden an.

In [ ]:
rolmean30.dropna(inplace=True)
mstlrol30 = MSTL(rolmean30['avgKwhConsum'], periods=[7, 14, 30, 365])
res30 = mstlrol30.fit()
res30.seasonal.head()


In [ ]:
ax = res30.plot()
plt.xticks(rotation=90)
plt.show()

In einer näheren Betrachtung der Saisonalitäten werden die tiefen sowie Spitzen erkennbar. Bei der wöchentlichen Ansicht, werden die ersten zwei Wochen ausgeblendet, da es sich dort um eine Ferienzeit handelt. Zur Eindordnung, der 14.1.2021 ist ein Donnerstag. Demzufolge ist der 16.1/17.1 das Wochenende. Die wöchentliche Saisonalität zeigt, dass es jeweils am Donnerstag einen Einbruch des Stromverbrauches gibt. Dieser steigt bis zum Freitag wieder, auf den Samstag ebenfalls bevor er auf den Sonntag jeweils sinkt. Am Montag scheint sich jeweils der Peak des Stromverbrauches zu zeigen.<br>
Bei der Betrachtung des 14-Tages Zyklus sieht man, dass dieser im zwei Wochen Rhythms sinkt und steigt. Dasselbe gilt für den 30-Tages Zyklus. Jeweils gegen Ende des Monats sinkt der Verbrauch und Mitte des Monats wird am meisten Strom benötigt.

In [ ]:
fig, ax = plt.subplots(nrows=4, figsize=[10,10])
res.seasonal["seasonal_7"].iloc[14:7*6].plot(ax=ax[0])
ax[0].set_ylabel("seasonal_7")
ax[0].set_title("Weekly seasonality")

res.seasonal["seasonal_14"].iloc[14:14*4].plot(ax=ax[1])
ax[1].set_ylabel("seasonal_14")
ax[1].set_title("2-Weekly seasonality")

res.seasonal["seasonal_30"].iloc[:30*3].plot(ax=ax[2])
ax[2].set_ylabel("seasonal_30")
ax[2].set_title("Monthly seasonality")

res.seasonal["seasonal_365"].iloc[:365*3].plot(ax=ax[3])
ax[3].set_ylabel("seasonal_365")
ax[3].set_title("Yearly seasonality")
plt.tight_layout()

Nachstehend noch der Vergleich in den Sommermonaten 2021. Die Wöchentliche Saisonalität ist nun deutlich ausgeprägter. Zur Einordnung der 19.6/20.6 sind Samstag und Sonntag. Im Gegensatz zu den Wintermonaten, bricht der Stromverbrauch am Mittwoch ein und nicht mehr am Donnerstag. Klar zu erkennen ist, dass am Wochenende am wenigsten Strom verbraucht wird. Das 14-Tage Intervall zeigt kaum mehr eine Saisonalität. Wobei das 30-Tage Intervall noch immer ausgeprägt ist.

In [ ]:
fig, ax = plt.subplots(nrows=3, figsize=[10,10])
res.seasonal["seasonal_7"].iloc[7*24:7*28].plot(ax=ax[0])
ax[0].set_ylabel("seasonal_7")
ax[0].set_title("Weekly seasonality")

res.seasonal["seasonal_14"].iloc[7*24:7*32].plot(ax=ax[1])
ax[1].set_ylabel("seasonal_14")
ax[1].set_title("2-Weekly seasonality")

res.seasonal["seasonal_30"].iloc[7*24:7*36].plot(ax=ax[2])
ax[2].set_ylabel("seasonal_30")
ax[2].set_title("Monthly seasonality")
plt.tight_layout()

Für die Vorhersage ist es wichtig zu Wissen ob es sich bei der Zeitreihe um eine stationäre oder nicht-stationäre Zeitreihe handelt. Dies aus dem Grund, weil stationäre Daten besser vorhergesagt werden können, da dort keine Brüche in Trends, Saisonalität oder Zyklen aufweisen. Also die Variance, Durchschnitt oder Autokorrelation sind konstant über die Zeit.<br>
Aus den vorhergehenden Analyse wissen wir bereits, dass es verschiedene Saisonalitäten gibt und sich die Autokorrelation über die Zeit ändert, je nach gewähltem Intervall. Der Augmented Dickey-Fuller Test hilft dabei festzustellen, ob es um stationäre oder nicht-stationäre Daten handelt. Die untstehende Funktion stammt von Daniel Benninger ([GitHub](https://github.com/sawubona-repo/BINA-FS24-WORK/blob/main/LB06-Regression%2BTimeSeries/Python/Python_JUPYTER_TIMESERIES_AirPassengers.ipynb), 2024)

In [ ]:
# define ADF stationarity test function for a time series
def stationarity_test(timeseries):
    # Augmented Dickey-Fuller (ADF) test
    print('Results of Augmented-Dickey-Fuller (ADF) Test\n')
    df_test = adfuller(timeseries)

    df_output = pd.Series(df_test[0:4], index = ['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in df_test[4].items():
        df_output['Critical Value (%s)' %key] = value

    print(df_output)

Der Test hat ergeben mit einem p-Value von 0.11, dass es sich um eine nicht-stationäre Zeitreihe handelt und die Variance nicht konstant über die Zeit ist. Auch die Teststatistik ist nicht kleiner wie die kritischen 1%, 5% oder 10% Intervalle. Damit werden die vorhergehenden Untersuchungen bestätigt und es handelt sich nicht um stationäre Daten.

In [ ]:
series = df_powerMeggen['avgKwhConsum'].values

stationarity_test(series)

Mit den First-Differences wird nun versucht aus den nicht stationären Daten, stationäre Daten zu produzieren. Der ADF Test scheint zu bestätigen, dass es sich nun um stationäre Daten handelt. Mit diesen wird nun versucht ein ARIMA-Model zu füllen.

In [ ]:
df_diff = df_powerMeggen
df_diff = df_diff.diff(periods = 7)
df_diff.dropna(inplace=True)
series = df_diff['avgKwhConsum'].values

stationarity_test(series)

### AutoARIMA Model
Für die Modellierung wird zunächst ein Training-Set und ein Test-Set erstellt. Es wird ein Set für die First-Differences und die echten Daten erstellt.

In [ ]:
df_trainPower = df_powerMeggen[df_powerMeggen.index < pd.to_datetime("2023-06-01", format='%Y-%m-%d')].copy()
df_trainPower.rename(columns={'avgKwhConsum': 'train'}, inplace=True)
df_trainPower.info()

In [ ]:
df_trainPowerDiff = df_diff[df_diff.index < pd.to_datetime("2023-06-01", format='%Y-%m-%d')].copy()
df_trainPowerDiff.rename(columns={'avgKwhConsum': 'train'}, inplace=True)
df_trainPowerDiff.info()

In [ ]:
df_testPower = df_powerMeggen[df_powerMeggen.index >= pd.to_datetime("2023-06-01", format='%Y-%m-%d')].copy()
df_testPower.rename(columns={'avgKwhConsum': 'test'}, inplace=True)
df_testPower.info()

In [ ]:
df_testPowerDiff = df_diff[df_diff.index >= pd.to_datetime("2023-06-01", format='%Y-%m-%d')].copy()
df_testPowerDiff.rename(columns={'avgKwhConsum': 'test'}, inplace=True)
df_testPowerDiff.info()

In der Grafik sind Trainingsdaten und Testdaten ersichtlich.

In [ ]:
plt.plot(df_trainPower, color = "black")
plt.plot(df_testPower, color = "red")
plt.title("Train/Test Trennung des Stromverbrauchs (echte Daten)")
plt.ylabel("Stromverbrauch kWh")
plt.xlabel('Tag')
plt.xticks(rotation=90)
sns.set()
plt.show()

In [ ]:
plt.plot(df_trainPowerDiff, color = "black")
plt.plot(df_testPowerDiff, color = "red")
plt.title("Train/Test Trennung des Stromverbrauchs (First-Differences)")
plt.ylabel("Stromverbrauch kWh")
plt.xlabel('Tag')
plt.xticks(rotation=90)
sns.set()
plt.show()

Nun wird das AutoARIMA Modell mit den Trainingsdaten bespielt. Zuerst mit einer Saisonalität von 7-Tagen.

In [ ]:
model = pm.auto_arima(df_trainPower,
                      m=7, seasonal=True,
                      error_action='ignore',
                      suppress_warnings=True,
                      stepwise=True, trace=True)
model.summary()

In [ ]:
modelDiff = pm.auto_arima(df_trainPowerDiff,
                      m=7, seasonal=True,
                      error_action='ignore',
                      suppress_warnings=True,
                      stepwise=True, trace=True)
modelDiff.summary()

Mit beiden Modellen werden nun die Vorhersagen getroffen und mit den Test Daten verglichen.

In [ ]:
pred = model.predict(n_periods=len(df_testPower), index=df_testPower)
predDiff = modelDiff.predict(n_periods=len(df_testPowerDiff), index=df_testPowerDiff)

Bei den echten Daten erkennt ARIMA den abwärts Trend aber nicht mehr die Saisonalität mit dem zunehmenden Stromverbrauch. Der 7-Tages Intervall ist zu kurz um den weiteren aufwärts Trend zu erkennen. Die Vorhersage passt nicht zu den echten Daten.

In [ ]:
plt.plot(df_trainPower, color = "black")
plt.plot(df_testPower, color = "red")
plt.plot(pred, color = "blue")
plt.title("Vergleich der Vorhersage mit den Testdaten (echte Daten)")
plt.ylabel("Stromverbrauch kWh")
plt.xlabel('Tag')
plt.xticks(rotation=90)
sns.set()
plt.show()

Mit den First Differences sieht es ähnlich aus. Zu Beginn wird erkannt, dass die Daten gering schwanken. Der Trend in der Vorhersage geht dann gegen 0 obwohl in der Realität die Schwankungen wieder zunehmen. Daher ist auch dieses Modell für die Vorhersage geeignet.

In [ ]:
plt.plot(df_trainPowerDiff, color = "black")
plt.plot(df_testPowerDiff, color = "red")
plt.plot(predDiff, color = "blue")
plt.title("Vergleich der Vorhersage mit den Testdaten (First-Differences)")
plt.ylabel("Stromverbrauch kWh")
plt.xlabel('Tag')
plt.xticks(rotation=90)
sns.set()
plt.show()

Der nächste Schritt wäre mit den echten Daten ein AutoARIMA Modell zu erstellen mit einem jährlichen Intervall. Doch mehrere Versuche sind gescheitert, da nach 40min (MacBook Pro mit M3 Pro) jeweils ein Memory Leak entstanden ist und zum Abbruch führte. Bis zu diesem Zeitpunkt konnten zwischen 6-7 Schritte von ARIMA durchgeführt werden für das passende Modell. Auf Goggle Colab brach der Prozess mehrfach nach ca. 2h ab mit demselben Fehler und der ungefähr gleichen Anzahl an Schritte. Recherchen haben gezeigt, dass es wohl an der Multi-Saisonalität liegt, die bereits in der vorhergehenden Analysen gefunden wurde. Damit kann ARIMA schlecht umgehen. <br>
Aus diesem Grund wurde TBATS als Alternative gefunden. Bei diesem Algorithmus können die Saisonalitäten mitgegeben werden. Nachfolgend folgt ein Versuch mit den echten Daten. Es werden die bekannten Saisonalitäten mitgegeben und der Trend auf True gesetzt, da sich über die drei Jahre ein negativer Trend angedeutet hat.

In [ ]:
#model = pm.auto_arima(df_trainPower,
#                      m=365, seasonal=True,
 #                     error_action='ignore',
  #                    suppress_warnings=True,
   #                   stepwise=True, trace=True)
#model.summary()

In [ ]:
estimator = TBATS(seasonal_periods=(7, 14, 30, 365), use_trend=True)
model_tbats = estimator.fit(df_trainPower)
forecast = model_tbats.forecast(steps=len(df_testPower))

Die Vorhersage sieht nun passender aus, da die Saisonalität erkannt wurde. Doch auch diese Kurve ist nicht passend und zu wenig gut um diese zu verwenden.

In [ ]:
df_forecast = pd.DataFrame(forecast, index=df_testPower.index)
plt.plot(df_trainPower, color = "black")
plt.plot(df_testPower, color = "red")
plt.plot(df_forecast, color = "blue")
plt.title("Vergleich der Vorhersage mit den Testdaten (TBATS)")
plt.ylabel("Stromverbrauch kWh")
plt.xlabel('Tag')
plt.xticks(rotation=90)
sns.set()
plt.show()

### Zusammenfassung
Es wurde eine mehrfache Saisonalität erkannt und das es sich um nicht stationäre Daten handelt. Dies macht die Vorhersage schwierig mit nur dieser Zeitreihe. Aus den vorhergehenden Anaylsen konnte zudem erkannt werden das externe Faktoren (hauptsächlich Temperatur) einen Einfluss auf den Stromverbrauch haben. Die hier verwendeten Algorithmen unterstützen keine Multivariate Modelle. AutoARIMA hat sich als nicht geeignet herausgestellt aufgrund der Multisaisonalität. TBATS kommt der echten Welt am nächsten aber ebenfalls nicht genügend. Zumindest wird dort die jährliche Saisonalität erkannt.

## Regression Model Stromverbrauch und Temperatur
Da ein Zusammenhang zwischen dem Stromverbrauch und der Temperatur erkannt wurde, wird versucht ein Regression Model zu erstellen. Dazu werden wieder die Daten von Meggen verwendet.


In [ ]:
df_meggen = df_1063.copy()
df_meggen.info()

Die Daten werden nun in einem Scatterplot dargestellt um eine Tendenz zu finden. Es scheint einen negativen liniearen Zusammenhang zugeben. Dies haben die Korrelationen aus der früheren Analysen bereits angedeutet. Ein Outlier ist erkennbar gegen Unten Links.

In [ ]:
sns.scatterplot(df_meggen, x='avgTempDay', y='avgKwhConsum').set(title='Scatterplot zischen Temperatur und dem Stromverbrauch', xlabel='Temperatur Grad Celsius', ylabel='Stromverbrauch kWh')
plt.figsize = (10,5)
plt.show()

In [ ]:
power_train, power_test, temp_train, temp_test = train_test_split(df_meggen['avgKwhConsum'], df_meggen['avgTempDay'], train_size = 0.8, test_size = 0.2, random_state = 100)

In [ ]:
power_train.info()

In [ ]:
power_test.info()

In [ ]:
temp_train.info()

In [ ]:
temp_test.info()

Das Regression-Model wird nach der Vorlage von Daniel Benninger erstellt ([GitHub](https://github.com/sawubona-repo/BINA-FS24-WORK/blob/main/LB06-Regression%2BTimeSeries/Python/Python_JUPYTER_Linear_REGRESSION_Advertising.ipynb), 2024). Das Modell gibt eine negative Steigung an und einen Intercept von 21.31.

In [ ]:
# Add a constant to get an intercept
temp_train_sm = sm.add_constant(temp_train)

# Fit the Regression Line using 'OLS' (ordinary least square)
lr = sm.OLS(power_train, temp_train_sm).fit()
lr.params

Anhand der Zusammenfassung wird bestätigt, dass mit einem tiefen p-Value die Koeffizienten einen signifikanten Einfluss auf den Stromverbrauch haben. Mit einem R-squared Wert von 0.753 werden die Daten gut beschrieben.

In [ ]:
print(lr.summary())

In der Grafik erkennt man, dass die lineare Linie gut zu den Punkten passt.

In [ ]:
plt.scatter(temp_train, power_train)
plt.plot(temp_train, 21.3065 + -0.5574*temp_train, 'r')

plt.title('Originale Daten und das lineare Model')
plt.figsize = (10,5)

# Set x-axis label
plt.xlabel('Temperatur Grad Celsius')
# Set y-axis label
plt.ylabel('Stromverbrauch kWh')

plt.show()

### Modell Evaluation
Um das Modell zu bestätigen, werden die Residiuale analysiert um die Zuverlässigkeit zu prüfen. Dazu wird die Verteilung der Abweichungen von der linearen Linie zu den realen Punkten analysiert. Die Verteilung ist normalverteilt mit einem Durschnitt von 0. Es gibt eine Aussnahme links aussen, diese stammt wohl von dem erkannten Outlier und kann ignoriert werden.

In [ ]:
power_train_pred = lr.predict(temp_train_sm)
res = (power_train - power_train_pred)

In [ ]:
plt.figure()
sns.displot(res, bins = 15, kde=True)

plt.title('Model Evaluation: Verteilung der Error Terms', fontsize = 15)
plt.xlabel('power_train - power_train_pred', fontsize = 15)         # X-label

plt.figsize = (10,5)
plt.show()

Die Residuale zeigen, dass die Daten gut beschrieben werden durch das lineare Modell. Im tieferen Temperaturen Bereich gibt es noch eine grössere Streuung auf als im höheren Bereich. Da gibt es noch Faktoren, die nicht durch dieses Model beschrieben werden können.

In [ ]:
plt.scatter(temp_train,res)

plt.title('Model Evaluation: Residual Patterns', fontsize = 15)
plt.ylabel('power_train - power_train_pred', fontsize = 15)         # Y-label

plt.figsize = (10,5)
plt.show()

### Vorhersagen basierend auf dem Test Set
Mit dem evaluierten Model können nun die Vorhersagen getestet werden. Der RMSE Wert sowie der R-squared Wert deuten auf ein gut passendes Modell hin.

In [ ]:
# Add a constant to X_test
temp_test_sm = sm.add_constant(temp_test)

# Predict the y values corresponding to X_test_sm
power_pred = lr.predict(temp_test_sm)


In [ ]:
#Returns the mean squared error (RMSE); we'll take a square root
RMSE = np.sqrt(mean_squared_error(power_test, power_pred))
print('Root Mean Squared Error (RMSE): ', RMSE)

In [ ]:
r_squared = r2_score(power_test, power_pred)
print('R-squared: ',r_squared)

Bei dieser Evaluation deutet sich an, dass ein lineares Model wohl nicht am besten die Realität widerspiegelt und lineare Linie underfitted ist. Es könnte in die Richtung einer polynomiale Regression gehen.

In [ ]:
plt.scatter(temp_test, power_test)
plt.plot(temp_test, 21.3065 + -0.5574* temp_test, 'r')

plt.title('Model Evaluation: Visualisierung der Passform des Modells auf die Testdaten', fontsize = 15)

plt.ylabel('Stromverbrauch kWh')         # Y-label

plt.xlabel('Temperatur Grad Celsius')            # X-label

plt.figsize = (10,5)
plt.show()

### Modell anwenden
Mit der Definition einer Funktion, können nun Vorhersagen basierend auf einem Input getroffen werden.

In [ ]:
def lr_model_prediction (Xarg):
    intercept = 21.3065
    coeff_X = -0.5574

    result = intercept + coeff_X * Xarg
    return result

In [ ]:
tmp = -2
print('Stromverbrauch Vorhersage:\t',lr_model_prediction(tmp),'kWh\nbei Temperatur: \t\t\t', tmp, 'Grad Celsius')

### Zusammenfassung
Generell kann die Aussage getroffen werden, dass für jedes Grad wärmer der Stromverbrauch um ca. 0.5 kWh abnimmt, in dieser linearen Betrachtung. Aufgrund der Residuale und der letzten Test Grafik scheint es noch weitere Faktoren zu geben, die den Stromverbrauch beeinflussen. Ebenfalls scheint es grafisch nach einem Underfitting auszusehen.
## Zusammenfassung Forecasting
Eine Vorhersage alleine basierend auf der Zeitreihe des Stromverbrauches ist nicht sinnvoll mit den angewandten Algorithmen. Ein lineares Modell scheint gut zu passen. Die tiefere Betrachtung zeigt aber eine Tendenz des Underfitting. Als nächster Schritt würde sich ein Long Short-Term Memory LSTM Modell anbieten, welches auf dem recurrent neural network RNN basiert. In diesem können verschiedene Inputs geliefert werden um einen Output zu liefern. Dies wird nicht weiter in dieser Arbeit behandelt.



# Fazit
## Datenqualität & Auswertungen
Die gesammelten Daten lassen eine Auswertung in die angestrebte Richtung zu. Besonders die Meteo-Daten sind zwar kompliziert in der Handhabung, lassen aber viele Rückschlüsse und Auswertungen zu. Auch die demografischen Daten des Bundes sind sehr ausführlich, wenn auch teilweise nicht top aktuell. Die Smartmeter Daten stellen jedoch die Schwachstelle dar. Die Daten weisen Ungenauigkeiten und offensichtliche Fehler auf. Diese fehlerhaften Datensätze ziehen sich in einigen Fällen über mehrere Monate hinweg, was das Herausfiltern und Auffüllen der Daten sinnlos macht. 

Da die Daten nur für Gemeinden im Kanton Luzern zur Verfügung stehen, ist das Datenset betreffend der Ortschaften stark begrenzt und fehlerhafte Daten engen es noch weiter ein. Um aussagekräftige Zusammenhänge zwischen Stromverbrauch, demografischen Daten und Meteo-Daten ziehen zu können, ist das Datenset zu klein und geografisch zu stark begrenzt. Weiter wäre es interessant, wenn die Smart Meter auch Angaben über den eingespeisten Strom machen würden. Damit wären in Verbindung mit den installierten Solaranlagen weitere Auswertungen möglich, ob das Potenzial ausgeschöpft wird. Leider fehlen aktuell Informationen, über den tatsächlich mit den installierten Solaranlagen produzierten Storm. 

Weiter wäre eine Kategorisierung der Smartmeter Daten nach privat und gewerblich sinnvoll. Zwar werden Grossverbraucher herausgefiltert, der Vergleich von Anzahl Smartmeter mit Anzahl Haushalten zeigt jedoch, dass je nach Gemeinde auch zahlreiche Messungen aus dem Gewerbe vorhanden sind. Dadurch sind keine separaten Auswertungen und Trendanalysen möglich.

## Persönliches Fazit
Wir haben uns für das Projekt eine zu breite Datenbasis vorgenommen. Dies machte einerseits das Aufbereiten und Analysieren der Daten sehr aufwendig, andererseits brachte es auch unsere IDEs teilweise ans Limit und brauchte jeweils lange für eine Durchführung. Besonders die Meteo-Daten waren sehr zeitintensiv und die Smartmeter Daten haben etwas enttäuscht. Einerseits durch die schlechte Datenqualität, andererseits dadurch, dass nur die PLZ zur Verfügung stand. Das eindeutige Mapping von PLZ auf die BFS-ID war aufwendig und musste von Hand vorgenommen werden. Da das Datenset auf den Kanton Luzern beschränkt ist, war dies möglich, jedoch wäre es bei einer Ausweitung auf die ganze Schweiz sehr unpraktisch. Hier empfiehlt es sich für die CKW AG, die Datensätze in Zukunft mit BFS-ID zur Verfügung zu stellen, da auch sämtliche anderen Datensätze damit arbeiten.

Insofern waren die Ergebnisse der Auswertungen im Hinblick auf die Fragestellung und die breite Datenbasis eher ernüchternd und aussagekräftige Zusammenhänge konnten keine festgestellt werden und für weitere, tiefere Auswertungen fehlte aufgrund der aufwendigen Aufbereitung der breiten Datenbasis die Zeit. Die ersten Versuche mit ML haben aber gezeigt, dass hier potenzial besteht, anhand der verschiedenen Parametern der Meteo-Daten den Stromverbrauch vorherzusagen.
